# This Notebook demonstrate the different version of dense mask transfer

Check the sparse points projection with depth map (3dscanner origin)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import time
import numpy as np
import cv2
import json
import glob
import random
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np 
from collections import namedtuple
from pycocotools import mask as mask_utils
from PIL import Image
from scipy.spatial import Delaunay
from tqdm.notebook import tqdm

In [ ]:
from helpers_cal import *
from helpers_masktransfer import *
from helpers_analysis import *

Core Data Path

In [ ]:
# COLMAP Path
colmap_scripts_path = "/userdata/chung-yu.wei/colmap/scripts/python"
if colmap_scripts_path not in sys.path:
    sys.path.append(colmap_scripts_path)

# Core data Paths 
base_path = "/userdata/chung-yu.wei/3DRealCar/data/2024_06_04_13_44_39/"
sparse_model_path = os.path.join(base_path, "colmap_processed/sparse/0")
rgb_images_path = os.path.join(base_path, "colmap_processed/input")
masks_path = os.path.join(base_path, "colmap_processed/masks/sam")

# json component path
component_json_path = os.path.join(base_path, "panel_seg/mask2former_prod_chunk_1_0_0/chunks/0.json")

# Depth map path
depth_maps_path = os.path.join(base_path, "3dscanner_origin")

import read_write_model
cameras, images, points3D = read_write_model.read_model(path=sparse_model_path, ext=".bin")
print("COLMAP model loaded successfully.")

In [ ]:
image_id_to_rgb_path = {}
image_id_to_depth_path = {} # for dense part
image_id_to_mask_path = {} # New dictionary for mask paths

# Iterate through all images in the Colmap model to build paths
for img_id, img_data in images.items():
    image_name = img_data.name
    try:
        base_name, ext = os.path.splitext(image_name) # e.g., ('frame_00025', '.jpg')
        number_part = base_name.split('_')[-1]      # e.g., '00025'
        
        # Map to RGB image path
        rgb_path = os.path.join(rgb_images_path, image_name)
        if os.path.exists(rgb_path):
            image_id_to_rgb_path[img_id] = rgb_path
            
        # Map to depth map path
        depth_filename = f"depth_{number_part}.png"
        depth_path = os.path.join(depth_maps_path, depth_filename)
        if os.path.exists(depth_path):
            image_id_to_depth_path[img_id] = depth_path

        # Map to vehicle mask path (assuming .jpg format from your screenshot)
        mask_filename = f"{base_name}.jpg" # e.g., 'frame_00025.jpg'
        mask_path = os.path.join(masks_path, mask_filename)
        if os.path.exists(mask_path):
            image_id_to_mask_path[img_id] = mask_path

    except Exception as e:
        print(f"Error processing filename {image_name}: {e}")

print(f"Successfully created RGB path mapping for {len(image_id_to_rgb_path)} images.")
print(f"Successfully created depth path mapping for {len(image_id_to_depth_path)} images.")
print(f"Successfully created mask path mapping for {len(image_id_to_mask_path)} images.")


In [ ]:
# Find image IDs that have both RGB and Mask files
valid_rgb_ids = set(image_id_to_rgb_path.keys())
valid_mask_ids = set(image_id_to_mask_path.keys())
valid_image_ids = list(valid_rgb_ids & valid_mask_ids)

### Visualization of Sparse point projection (This is from part 2.2)

In [ ]:
images_to_visualize_ids = []
images_to_visualize_ids = random.sample(valid_image_ids, 3)

if not images_to_visualize_ids:
    print("Error: Could not find any valid images with both RGB and Mask files for visualization.")
else:
    # Create a figure with subplots for each image 
    num_images = len(images_to_visualize_ids)
    fig1, axes1 = plt.subplots(1, num_images, figsize=(8 * num_images, 8))
    if num_images == 1: # Handle the case of a single image
        axes1 = [axes1]

    print(f"Visualizing sparse points for Image IDs: {images_to_visualize_ids}")

    # Loop through, process, and plot each image 
    for i, image_id in enumerate(images_to_visualize_ids):
        ax = axes1[i]
        image_name = images[image_id].name
        
        # Load data
        source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[image_id]), cv2.COLOR_BGR2RGB)
        source_mask = cv2.imread(image_id_to_mask_path[image_id], cv2.IMREAD_GRAYSCALE)

        if source_rgb is not None and source_mask is not None:
            # Filter keypoints
            keypoints_on_car = []
            for point in images[image_id].xys:
                u, v = int(point[0]), int(point[1])
                h, w = source_mask.shape
                if 0 <= v < h and 0 <= u < w and source_mask[v, u] > 128:
                    keypoints_on_car.append((u, v))
            
            # Visualize
            ax.imshow(source_rgb)
            if keypoints_on_car:
                points_array = np.array(keypoints_on_car)
                ax.scatter(points_array[:, 0], points_array[:, 1], s=15, c='purple', alpha=0.8, edgecolors='black', linewidths=0.5)
            ax.set_title(f"Result for: {image_name}\n({len(keypoints_on_car)} points on mask)", fontsize=12)
        else:
            ax.set_title(f"Error loading data for\n{image_name}")
        ax.axis('off')

    # # Display the entire figure
    # fig1.suptitle("Baseline Visualization: Sparse Keypoint Distribution on Vehicle Surface", fontsize=20, y=1.03)
    # plt.tight_layout()
    # plt.show()

print("\n Visualize the sparse points complete.")

In [ ]:
# Randomly select a source and target image
source_image_id_proj = None
target_image_id_proj = None

source_image_id_proj, target_image_id_proj = random.sample(valid_image_ids, 2)
print(f"Randomly selected source image: {images[source_image_id_proj].name} (ID: {source_image_id_proj})")
print(f"Randomly selected target image: {images[target_image_id_proj].name} (ID: {target_image_id_proj})")

if source_image_id_proj and target_image_id_proj:
    # Filter sparse keypoints on the source image that have a 3D point
    source_mask_proj = cv2.imread(image_id_to_mask_path[source_image_id_proj], cv2.IMREAD_GRAYSCALE)
    
    # Store keypoints that are on the car AND have a valid 3D point ID
    keypoints_for_projection = []
    if source_mask_proj is not None:
        for idx, point in enumerate(images[source_image_id_proj].xys):
            if images[source_image_id_proj].point3D_ids[idx] != -1: # Check for valid 3D ID
                u, v = int(point[0]), int(point[1])
                h, w = source_mask_proj.shape
                if 0 <= v < h and 0 <= u < w and source_mask_proj[v, u] > 128:
                    keypoints_for_projection.append({'uv': (u, v), 'original_index': idx})
    
    print(f"Found {len(keypoints_for_projection)} sparse keypoints on the source vehicle that can be projected.")

    if not keypoints_for_projection:
        print("Error: No projectable sparse keypoints found on the vehicle mask.")
    else:
        # Select one keypoint and perform the 2D -> 3D -> 2D projection 
        random_keypoint_data = random.choice(keypoints_for_projection)
        u_source, v_source = random_keypoint_data['uv']
        point_index = random_keypoint_data['original_index']
        
        print(f"Randomly selected source keypoint at: ({u_source}, {v_source})")

        # The core of the sparse method: Find the 3D point id using images.bin
        point_3d_id = images[source_image_id_proj].point3D_ids[point_index]
        point_3d_xyz = points3D[point_3d_id].xyz
        
        # Project the 3D point to the target image (Use the helper functions)
        target_image_model = images[target_image_id_proj]
        target_camera_model = cameras[target_image_model.camera_id]
        projected_point = project_point(point_3d_xyz, target_camera_model, target_image_model)

        # Visualize the projection result
        source_rgb_proj = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_image_id_proj]), cv2.COLOR_BGR2RGB)
        target_rgb_proj = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_image_id_proj]), cv2.COLOR_BGR2RGB)

        fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        
        ax1.imshow(source_rgb_proj)
        ax1.scatter(u_source, v_source, s=150, c='magenta', marker='X')
        ax1.set_title(f'Source: {images[source_image_id_proj].name}\nSparse Keypoint Selected', fontsize=16)
        ax1.axis('off')
        
        ax2.imshow(target_rgb_proj)
        if projected_point:
            u_proj, v_proj = projected_point
            ax2.scatter(u_proj, v_proj, s=150, c='magenta', marker='X')
            ax2.set_title(f'Target: {images[target_image_id_proj].name}\nSparse Projected Point', fontsize=16)
        else:
            ax2.set_title(f'Target: {images[target_image_id_proj].name}\nPoint could not be projected', fontsize=16)
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()

print("\nAll baseline experiments complete.")

## Phase1: Sparse point been projected using depth map (Compared with ground truth)

In [ ]:
# Loop to generate 5 different comparison examples
for i in range(5):
    print(f"--- Running Comparison Example {i+1} of 5 ---")
    
    # Randomly select a source and target image
    source_image_id_proj = None
    target_image_id_proj = None

    # Ensure we have images that also have corresponding depth maps
    valid_ids_with_depth = list(set(valid_image_ids) & set(image_id_to_depth_path.keys()))

    if len(valid_ids_with_depth) < 2:
        print("Error: Could not find at least 2 images with RGB, Mask, and Depth data for comparison.")
        break # Exit the loop if data is insufficient
    
    source_image_id_proj, target_image_id_proj = random.sample(valid_ids_with_depth, 2)
    print(f"Randomly selected source image: {images[source_image_id_proj].name} (ID: {source_image_id_proj})")
    print(f"Randomly selected target image: {images[target_image_id_proj].name} (ID: {target_image_id_proj})")

    if source_image_id_proj and target_image_id_proj:
        source_mask_proj = cv2.imread(image_id_to_mask_path[source_image_id_proj], cv2.IMREAD_GRAYSCALE)

        keypoints_for_projection = []
        if source_mask_proj is not None:
            for idx, point in enumerate(images[source_image_id_proj].xys):
                if images[source_image_id_proj].point3D_ids[idx] != -1:
                    u, v = int(point[0]), int(point[1])
                    h, w = source_mask_proj.shape
                    if 0 <= v < h and 0 <= u < w and source_mask_proj[v, u] > 128:
                        keypoints_for_projection.append({'uv': (u, v), 'original_index': idx})

        if not keypoints_for_projection:
            print("Warning: No projectable sparse keypoints found on the vehicle mask for this image. Skipping.")
            continue

        print(f"Found {len(keypoints_for_projection)} sparse keypoints on the source vehicle that can be projected.")
        
        random_keypoint_data = random.choice(keypoints_for_projection)
        u_source, v_source = random_keypoint_data['uv']
        point_index = random_keypoint_data['original_index']

        print(f"Randomly selected source keypoint at: ({u_source}, {v_source})")

        source_image_model = images[source_image_id_proj]
        source_camera_model = cameras[source_image_model.camera_id]
        target_image_model = images[target_image_id_proj]
        target_camera_model = cameras[target_image_model.camera_id]

        # --- Method 1: Sparse Projection ---
        point_3d_id = images[source_image_id_proj].point3D_ids[point_index]
        point_3d_xyz_sparse = points3D[point_3d_id].xyz
        projected_point_sparse = project_point(point_3d_xyz_sparse, target_camera_model, target_image_model)

        # --- Method 2: Dense Projection ---
        projected_point_dense = None
        source_rgb_proj_for_size = cv2.imread(image_id_to_rgb_path[source_image_id_proj])
        rgb_h, rgb_w, _ = source_rgb_proj_for_size.shape
        source_depth_map = cv2.imread(image_id_to_depth_path[source_image_id_proj], cv2.IMREAD_UNCHANGED)

        if source_depth_map is not None:
            if source_depth_map.shape[0] != rgb_h or source_depth_map.shape[1] != rgb_w:
                source_depth_map = cv2.resize(source_depth_map, (rgb_w, rgb_h), interpolation=cv2.INTER_NEAREST)

            depth_in_meters, error_msg = get_depth_from_map(u_source, v_source, source_depth_map, scale_factor=1000.0)

            if depth_in_meters:
                point_3d_xyz_dense = back_project_point(u_source, v_source, depth_in_meters, source_camera_model, source_image_model)
                if point_3d_xyz_dense is not None:
                    projected_point_dense = project_point(point_3d_xyz_dense, target_camera_model, target_image_model)
            else:
                print(f"Could not get valid depth for dense projection: {error_msg}")
        else:
            print("Could not load the depth map file.")

        # --- Visualization ---
        source_rgb_proj = cv2.cvtColor(source_rgb_proj_for_size, cv2.COLOR_BGR2RGB)
        target_rgb_proj = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_image_id_proj]), cv2.COLOR_BGR2RGB)

        fig_comp, (ax_comp1, ax_comp2) = plt.subplots(1, 2, figsize=(20, 10))
        fig_comp.suptitle(f"Comparison Example {i+1}", fontsize=20, y=1.02)
        
        ax_comp1.imshow(source_rgb_proj)
        ax_comp1.scatter(u_source, v_source, s=250, c='cyan', marker='X')
        ax_comp1.set_title(f'Source: {source_image_model.name}\nKeypoint Selected', fontsize=16)
        ax_comp1.axis('off')

        ax_comp2.imshow(target_rgb_proj)
        ax_comp2.set_title(f'Target: {target_image_model.name}\nProjection Comparison', fontsize=16)
        
        plot_handles = []

        if projected_point_sparse:
            u_proj_s, v_proj_s = projected_point_sparse
            sparse_plot = ax_comp2.scatter(u_proj_s, v_proj_s, s=150, c='magenta', marker='X', label='Sparse (COLMAP)')
            plot_handles.append(sparse_plot)

        if projected_point_dense:
            u_proj_d, v_proj_d = projected_point_dense
            dense_plot = ax_comp2.scatter(u_proj_d, v_proj_d, s=200, c='lime', marker='+', label='Dense (Depth Map)')
            plot_handles.append(dense_plot)
            
        # CALCULATE AND DISPLAY THE DISTANCE
        if projected_point_sparse and projected_point_dense:
            p_sparse = np.array(projected_point_sparse)
            p_dense = np.array(projected_point_dense)
            
            # Calculate Euclidean distance
            distance = np.linalg.norm(p_sparse - p_dense)
            
            print(f"  > Pixel distance between points: {distance:.2f} pixels")
            
            # Display distance on the plot
            ax_comp2.text(15, 35, f'Error: {distance:.2f} pixels', 
                          style='italic', fontsize=14,
                          bbox={'facecolor': 'white', 'alpha': 0.7, 'pad': 5})

        if not plot_handles:
             ax_comp2.set_title(f'Target: {target_image_model.name}\nPoint could not be projected', fontsize=16)
        else:
            ax_comp2.legend(handles=plot_handles)

        ax_comp2.axis('off')

        plt.tight_layout()
        plt.show()
        print("\n" + "="*50 + "\n")

print("\nAll comparison experiments complete.")

Visualization of 5 biggest projection error

In [ ]:
# source_id, target_id = random.sample(valid_image_ids, 2)

# analysis_results = analyze_reprojection_error(
#     source_id,
#     target_id,
#     images,
#     cameras,
#     points3D,
#     image_id_to_mask_path,
#     image_id_to_rgb_path,
#     image_id_to_depth_path
# )

# # If the analysis was successful, create the plot
# if analysis_results:
#     plot_worst_offender(
#         source_id,
#         target_id,
#         analysis_results["worst_offender"],
#         images,
#         image_id_to_rgb_path
#     )

## Phase2: Baseline Dense Mask transfer

In [ ]:
# 1. Load component data
image_name_to_anns, cat_id_to_name = load_component_data(component_json_path)

In [ ]:
# 2. Loop to run the analysis 5 times, plotting each result separately
for i in range(5):
    print(f"\n{'='*20} Running Mask Transfer Example {i+1} of 5 {'='*20}\n")

    source_id, target_id, source_component_name = None, None, None
    attempt = 0
    while source_id is None and attempt < 200:
        potential_source_id = random.choice(valid_image_ids)
        normalized_colmap_name = os.path.splitext(os.path.basename(images[potential_source_id].name))[0]
        if normalized_colmap_name in image_name_to_anns:
            ann_for_img = image_name_to_anns[normalized_colmap_name]
            random_ann = random.choice(ann_for_img)
            source_component_name = cat_id_to_name[random_ann['category_id']]
            source_id = potential_source_id
            target_id = random.choice([i for i in valid_image_ids if i != source_id])
        attempt += 1

    if source_id and target_id and source_component_name:
        print(f"--- Starting Component Transfer ---")
        print(f"Source Image: {images[source_id].name} (ID: {source_id})")
        print(f"Target Image: {images[target_id].name} (ID: {target_id})")
        print(f"Component to Transfer: '{source_component_name}'")
        
        source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
        source_h, source_w, _ = source_rgb.shape

        source_mask, error_msg = get_component_mask_robust(
            images[source_id].name,
            source_component_name,
            image_name_to_anns,
            cat_id_to_name,
            source_h,
            source_w
        )

        if source_mask is not None:
            # --- Calling the ULTIMATE transfer function ---
            transferred_mask = transfer_mask_v3(
                source_id,
                target_id,
                source_mask,
                source_component_name,
                images,
                cameras,
                image_id_to_rgb_path,
                image_id_to_depth_path,
                image_id_to_mask_path
            )

            # --- Visualization for this single example ---
            target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
            source_overlay = source_rgb.copy()
            source_overlay[source_mask > 0] = [255, 0, 150]

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
            fig.suptitle(f"Mask Transfer Example {i+1}", fontsize=20, y=0.98)
            ax1.imshow(source_overlay)
            ax1.set_title(f"Source: Original '{source_component_name}' Mask", fontsize=16)
            ax1.axis('off')
            ax2.imshow(target_rgb)

            if transferred_mask is not None:
                target_overlay = target_rgb.copy()
                roi = target_overlay[transferred_mask > 0]
                blended_roi = (roi * 0.5 + np.array([0, 255, 150], dtype=np.uint8) * 0.5).astype(np.uint8)
                target_overlay[transferred_mask > 0] = blended_roi
                ax2.imshow(target_overlay)
                ax2.set_title(f"Target: Transferred '{source_component_name}' Mask", fontsize=16)
            else:
                ax2.set_title(f"Target: Mask transfer failed", fontsize=16)
            ax2.axis('off')

            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.show()
        else:
            print(f"Error getting component mask: {error_msg}")
    else:
        print("Could not find a valid image and component combination for this run. Skipping.")

print("\nAll 5 mask transfer examples are complete.")

## Phase3: Version 1 Dense Mask Transfer

In [ ]:
# for i in range(5): # Loop for 5 examples
#     print(f"\n{'='*20} Running Conditional Evaluation Example {i+1} of 5 {'='*20}\n")

#     source_id, target_id, component_name = None, None, None
#     attempt = 0
#     while source_id is None and attempt < 200:
#         potential_source_id = random.choice(valid_image_ids)
#         normalized_colmap_name = os.path.splitext(os.path.basename(images[potential_source_id].name))[0]
#         if normalized_colmap_name in image_name_to_anns:
#             ann_for_img = image_name_to_anns[normalized_colmap_name]
#             random_ann = random.choice(ann_for_img)
#             component_name = cat_id_to_name[random_ann['category_id']]
#             source_id = potential_source_id
#             target_id = random.choice([i for i in valid_image_ids if i != source_id])
#         attempt += 1

#     if source_id:
#         print(f"--- Starting Component Transfer ---\nSource: {images[source_id].name} | Target: {images[target_id].name} | Component: '{component_name}'")
        
#         # Get source image dimensions
#         source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
#         source_h, source_w, _ = source_rgb.shape
        
#         # Use the robust mask getter
#         source_mask, _ = get_component_mask_robust(images[source_id].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)
        
#         if source_mask is not None:
#             # *** THIS IS THE UPDATED LINE ***
#             # We now pass the required global data to the function
#             transferred_mask = transfer_mask_v1(
#                 source_id,
#                 target_id,
#                 source_mask,
#                 images,
#                 cameras,
#                 image_id_to_rgb_path,
#                 image_id_to_depth_path
#             )
            
#             target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
#             target_h, target_w, _ = target_rgb.shape
#             target_gt_mask, _ = get_component_mask_robust(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name, target_h, target_w)

#             if target_gt_mask is not None:
#                 iou_score = calculate_iou(transferred_mask, target_gt_mask) if transferred_mask is not None else 0.0
#                 print(f"\n>>> IoU Score: {iou_score:.4f} <<<")
#                 fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
#                 fig.suptitle(f"IoU Evaluation for '{component_name}': {iou_score:.4f}", fontsize=24)
#                 ax1.imshow(source_rgb); ax1.imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.5); ax1.set_title("1. Source Mask"); ax1.axis('off')
#                 ax2.imshow(target_rgb); ax2.imshow(np.ma.masked_where(transferred_mask==0, transferred_mask), cmap='winter', alpha=0.5); ax2.set_title("2. Transferred Mask"); ax2.axis('off')
#                 ax3.imshow(target_rgb); ax3.imshow(np.ma.masked_where(target_gt_mask==0, target_gt_mask), cmap='autumn', alpha=0.5); ax3.set_title("3. Ground Truth Mask"); ax3.axis('off')
            
#             else:
#                 target_vehicle_mask = cv2.imread(image_id_to_mask_path.get(target_id, ''), cv2.IMREAD_GRAYSCALE)
#                 spread_rate = 0.0
#                 if transferred_mask is not None and target_vehicle_mask is not None:
#                     spread_rate = calculate_projection_spread(transferred_mask, target_vehicle_mask)
#                     print(f"\n>>> Projection Spread Rate: {spread_rate:.2%} <<<")
                
#                 fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
#                 fig.suptitle(f"Mask Transfer for '{component_name}' (Spread Rate: {spread_rate:.2%})", fontsize=24)
#                 source_overlay = source_rgb.copy(); source_overlay[source_mask > 0] = [255, 0, 150]; ax1.imshow(source_overlay); ax1.set_title("Source Mask"); ax1.axis('off')
#                 target_overlay = target_rgb.copy()
#                 if transferred_mask is not None and target_vehicle_mask is not None:
#                     in_vehicle = np.logical_and(transferred_mask, target_vehicle_mask); roi_in = target_overlay[in_vehicle]; blended_roi_in = (roi_in*0.5+np.array([0,255,150],dtype=np.uint8)*0.5).astype(np.uint8); target_overlay[in_vehicle] = blended_roi_in
#                     false_positives = np.logical_and(transferred_mask, np.logical_not(target_vehicle_mask)); roi_fp = target_overlay[false_positives]; blended_roi_fp = (roi_fp*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); target_overlay[false_positives] = blended_roi_fp
#                 ax2.imshow(target_overlay); ax2.set_title("Transferred Mask (Cyan=On Car, Red=Spread)"); ax2.axis('off')

#             plt.tight_layout(rect=[0, 0, 1, 0.95])
#             plt.show()
#     else:
#         print("Could not find a valid source image with a component to start the transfer. Skipping.")

# print("\nAll 5 conditional evaluation examples are complete.")

## Phase4: Version 2 dense mask transfer

In [ ]:
print("--- Pre-processing: Finding the best view for each component ---")

# Create a map: {component_name: [(area, image_id), ...]}
component_area_map = {}

# Create a reverse map to find image_id from normalized name
normalized_name_to_id = {
    os.path.splitext(os.path.basename(img.name))[0]: img_id
    for img_id, img in images.items()
}

# Iterate through all annotated images
for normalized_name, annotations in image_name_to_anns.items():
    if normalized_name in normalized_name_to_id:
        image_id = normalized_name_to_id[normalized_name]
        for ann in annotations:
            component_name = cat_id_to_name[ann['category_id']]
            
            # Decode mask and calculate area
            mask = mask_utils.decode(ann['segmentation'])
            area = np.sum(mask > 0)
            
            # Store the data
            if component_name not in component_area_map:
                component_area_map[component_name] = []
            component_area_map[component_name].append((area, image_id))

# Sort the lists for each component by area (largest first)
for component_name in component_area_map:
    component_area_map[component_name].sort(key=lambda x: x[0], reverse=True)

print("Pre-processing complete. Stored best views for all components.")

Illustration of missing values in depth map

In [ ]:
# # --- Main Execution Block ---

# print("--- Displaying 5 Examples of Missing Values in Depth Maps ---")

# # Ensure we have valid IDs to sample from
# if 'valid_image_ids' in locals() and valid_image_ids:
#     # Randomly select 5 unique image IDs to showcase
#     num_examples = 5
#     if len(valid_image_ids) < num_examples:
#         sample_ids = valid_image_ids
#     else:
#         sample_ids = random.sample(valid_image_ids, num_examples)

#     # Loop through the selected IDs and generate a plot for each
#     for i, image_id in enumerate(sample_ids):
#         image_name = images[image_id].name
#         depth_path = image_id_to_depth_path.get(image_id)

#         if depth_path and os.path.exists(depth_path):
#             # Load the depth map as-is (16-bit single channel)
#             depth_map = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)

#             if depth_map is not None:
#                 # --- Create the Visualization ---

#                 # 1. Create a mask of the missing pixels (where depth is 0)
#                 missing_mask = np.where(depth_map == 0, 255, 0).astype(np.uint8)

#                 # 2. For visualization, normalize the depth map to be visible as an 8-bit image
#                 # This makes the non-black areas easier to see
#                 depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

#                 # 3. Create the plot
#                 fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
#                 fig.suptitle(f"Depth Map Analysis for: {image_name} (Example {i+1})", fontsize=20)

#                 # Panel 1: Original Depth Map
#                 ax1.imshow(depth_map_normalized, cmap='gray')
#                 ax1.set_title("Original Depth Map (Normalized)", fontsize=16)
#                 ax1.axis('off')

#                 # Panel 2: Missing Values Highlighted
#                 ax2.imshow(missing_mask, cmap='gray')
#                 ax2.set_title("Missing Values (Highlighted in White)", fontsize=16)
#                 ax2.axis('off')

#                 plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#                 plt.show()
#             else:
#                 print(f"Warning: Could not load depth map for {image_name}")
#         else:
#             print(f"Warning: No valid depth map path found for {image_name}")
# else:
#     print("Error: `valid_image_ids` is not defined or is empty. Please run previous setup cells.")

### Now we focus on these components

In [ ]:
# Define the list of components you want to analyze
target_components = [
    "bonnet",
    "bumper_f/cover",
    "windshield_f",
    "headlamp_l_assy",
    "mirror_l_assy",
    "grille",
    "door_fl_assy"
]

Comparison of adding and not adding Z-buffer

In [ ]:
# # --- Transfer Function WITHOUT Z-Buffering ---
# def transfer_mask_no_zbuffer(source_id, target_id, source_component_mask):
#     """ Transfers a mask using a simple 'splat' method WITHOUT Z-buffering. """
#     print("--- Running: transfer_mask_no_zbuffer (Splat Method) ---")
#     target_rgb_for_size = cv2.imread(image_id_to_rgb_path[target_id]); target_h, target_w, _ = target_rgb_for_size.shape
#     source_rgb_for_size = cv2.imread(image_id_to_rgb_path[source_id]); source_h, source_w, _ = source_rgb_for_size.shape
#     source_depth_map_raw = cv2.imread(image_id_to_depth_path[source_id], cv2.IMREAD_UNCHANGED)
#     inpaint_mask = np.where(source_depth_map_raw == 0, 255, 0).astype(np.uint8)
#     source_depth_map = cv2.inpaint(source_depth_map_raw, inpaint_mask, 3, cv2.INPAINT_NS)
#     source_depth_map = cv2.resize(source_depth_map, (source_w, source_h), interpolation=cv2.INTER_NEAREST)
#     source_pixels_y, source_pixels_x = np.where(source_component_mask > 0)
#     source_pixels = np.vstack((source_pixels_x, source_pixels_y)).T
#     projected_pixels = []
#     source_image_model, source_camera_model = images[source_id], cameras[images[source_id].camera_id]
#     target_image_model, target_camera_model = images[target_id], cameras[images[target_id].camera_id]
#     for u, v in source_pixels:
#         depth, _ = get_depth_from_map(u, v, source_depth_map, scale_factor=1000.0)
#         if depth:
#             p_3d = back_project_point(u, v, depth, source_camera_model, source_image_model)
#             if p_3d is not None:
#                 p_2d_target = project_point(p_3d, target_camera_model, target_image_model)
#                 if p_2d_target:
#                     projected_pixels.append(p_2d_target)
#     if not projected_pixels: return None
#     transferred_mask_raw = np.zeros((target_h, target_w), dtype=np.uint8)
#     valid_points = np.round(projected_pixels).astype(int)
#     valid_indices = (valid_points[:, 0] >= 0) & (valid_points[:, 0] < target_w) & (valid_points[:, 1] >= 0) & (valid_points[:, 1] < target_h)
#     valid_points = valid_points[valid_indices]
#     transferred_mask_raw[valid_points[:, 1], valid_points[:, 0]] = 255
#     kernel = np.ones((5, 5), np.uint8)
#     closed_mask = cv2.morphologyEx(transferred_mask_raw, cv2.MORPH_CLOSE, kernel, iterations=2)
#     return closed_mask

# # --- Main Execution Block for Focused Comparison ---

# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running Z-Buffer Comparison for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")
    
#     source_id, target_id = None, None
    
#     # Find the best source view for this specific component
#     best_view_list = component_area_map.get(component_name)
#     if best_view_list:
#         source_id = best_view_list[0][1]
#         target_id = random.choice([img_id for img_id in valid_image_ids if img_id != source_id])
#     else:
#         print(f"Could not find a valid source view for '{component_name}'. Skipping.")
#         continue

#     if source_id:
#         print(f"--- Starting Comparison ---\nComponent: '{component_name}'\nSource: {images[source_id].name}\nTarget: {images[target_id].name}")
        
#         source_mask, error_msg = get_component_mask(images[source_id].name, component_name, image_name_to_anns, cat_id_to_name)
        
#         if source_mask is not None:
#             mask_no_z = transfer_mask_no_zbuffer(source_id, target_id, source_mask)
#             mask_with_z = transfer_mask_v1(source_id, target_id, source_mask)
            
#             source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
#             target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)

#             fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
#             fig.suptitle(f"Z-Buffer Comparison for '{component_name}'", fontsize=24, y=1.0)

#             ax1.imshow(source_rgb); ax1.imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.5)
#             ax1.set_title("1. Original Source Mask", fontsize=18); ax1.axis('off')

#             ax2.imshow(target_rgb)
#             if mask_no_z is not None:
#                 overlay_v1 = target_rgb.copy(); roi_v1 = overlay_v1[mask_no_z > 0]; blended_v1 = (roi_v1*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v1[mask_no_z > 0] = blended_v1; ax2.imshow(overlay_v1)
#             ax2.set_title("2. Result WITHOUT Z-Buffer", fontsize=18); ax2.axis('off')

#             ax3.imshow(target_rgb)
#             if mask_with_z is not None:
#                 overlay_v2 = target_rgb.copy(); roi_v2 = overlay_v2[mask_with_z > 0]; blended_v2 = (roi_v2*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v2[mask_with_z > 0] = blended_v2; ax3.imshow(overlay_v2)
#             ax3.set_title("3. Result WITH Z-Buffer (Correct)", fontsize=18); ax3.axis('off')
            
#             plt.tight_layout(rect=[0, 0, 1, 0.95])
#             plt.show()

# print("\nAll specific component comparisons for Z-Buffering are complete.")

Different point of view mask transfer combination (Specific Components)

In [ ]:
# --- 1. Pre-processing: Indexing and sorting all views for each component ---
print("--- Pre-processing: Indexing and sorting all views for each component ---")
component_to_views_map = {}
for normalized_name, annotations in image_name_to_anns.items():
    if normalized_name in normalized_name_to_id:
        image_id = normalized_name_to_id[normalized_name]
        for ann in annotations:
            component_name = cat_id_to_name[ann['category_id']]
            if component_name not in component_to_views_map:
                component_to_views_map[component_name] = []
            mask = mask_utils.decode(ann['segmentation'])
            area = np.sum(mask > 0)
            component_to_views_map[component_name].append((area, image_id))

# Sort each list by area in descending order
for name in component_to_views_map:
    component_to_views_map[name].sort(key=lambda x: x[0], reverse=True)
print("Pre-processing complete.")

In [ ]:
# # --- 2. Main Execution Block for Specific Components ---

# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running Multi-View Analysis for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")

#     source_a_id, source_b_id, target_id = None, None, None
    
#     # --- Logic to find the best two source views for the SPECIFIC component ---
    
#     # Check if the component is available and has at least 3 views
#     if component_name in component_to_views_map and len(component_to_views_map[component_name]) >= 3:
#         sorted_views = component_to_views_map[component_name]
        
#         # Source A is the best view (largest area)
#         source_a_id = sorted_views[0][1]
#         # Source B is the second-best view
#         source_b_id = sorted_views[1][1]
        
#         # Target C is a random choice from the rest of the available views
#         remaining_views = [view[1] for view in sorted_views[2:]]
#         target_id = random.choice(remaining_views)
#     else:
#         print(f"Could not find at least 3 views for '{component_name}'. Skipping.")
#         continue # Move to the next component in the list

#     if source_a_id:
#         print(f"Found valid triplet for component '{component_name}':")
#         print(f"  - Source A (Best View): {images[source_a_id].name} (ID: {source_a_id})")
#         print(f"  - Source B (2nd Best): {images[source_b_id].name} (ID: {source_b_id})")
#         print(f"  - Target C (Random):  {images[target_id].name} (ID: {target_id})")

#         source_a_mask, _ = get_component_mask(images[source_a_id].name, component_name, image_name_to_anns, cat_id_to_name)
#         source_b_mask, _ = get_component_mask(images[source_b_id].name, component_name, image_name_to_anns, cat_id_to_name)

#         if source_a_mask is not None and source_b_mask is not None:
#             # --- Perform both projections to the target view ---
#             print("\n--- Projecting from Source A to Target ---")
#             transfer_a_to_c = transfer_mask_v2(source_a_id, target_id, source_a_mask, component_name)
            
#             print("\n--- Projecting from Source B to Target ---")
#             transfer_b_to_c = transfer_mask_v2(source_b_id, target_id, source_b_mask, component_name)

#             # --- Calculate the Intersection ---
#             intersection_mask = None
#             if transfer_a_to_c is not None and transfer_b_to_c is not None:
#                 intersection_mask = cv2.bitwise_and(transfer_a_to_c, transfer_b_to_c)
#                 print("\nSuccessfully calculated the intersection of the two projections.")

#             # --- 4-Panel Visualization ---
#             source_a_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_a_id]), cv2.COLOR_BGR2RGB)
#             source_b_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_b_id]), cv2.COLOR_BGR2RGB)
#             target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)

#             fig, axes = plt.subplots(2, 2, figsize=(20, 20))
#             fig.suptitle(f"Multi-View Consistency Check for '{component_name}'", fontsize=24, y=0.95)
#             ((ax1, ax2), (ax3, ax4)) = axes

#             # Panel 1: Source A (in Yellow)
#             source_a_overlay = source_a_rgb.copy()
#             roi_a_s = source_a_overlay[source_a_mask > 0]; blended_a_s = (roi_a_s*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); source_a_overlay[source_a_mask > 0] = blended_a_s
#             ax1.imshow(source_a_overlay); ax1.set_title("1. Source A (Best View)"); ax1.axis('off')

#             # Panel 2: Source B (in Yellow)
#             source_b_overlay = source_b_rgb.copy()
#             roi_b_s = source_b_overlay[source_b_mask > 0]; blended_b_s = (roi_b_s*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); source_b_overlay[source_b_mask > 0] = blended_b_s
#             ax2.imshow(source_b_overlay); ax2.set_title("2. Source B (2nd Best View)"); ax2.axis('off')
            
#             # Panel 3: Overlaid Projections
#             target_overlay_3 = target_rgb.copy()
#             if transfer_a_to_c is not None:
#                 roi_a = target_overlay_3[transfer_a_to_c > 0]; blended_a = (roi_a*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); target_overlay_3[transfer_a_to_c > 0] = blended_a
#             if transfer_b_to_c is not None:
#                 roi_b = target_overlay_3[transfer_b_to_c > 0]; blended_b = (roi_b*0.5+np.array([0,0,255],dtype=np.uint8)*0.5).astype(np.uint8); target_overlay_3[transfer_b_to_c > 0] = blended_b
#             ax3.imshow(target_overlay_3); ax3.set_title("3. Overlaid Projections (A->C in Red, B->C in Blue)"); ax3.axis('off')

#             # Panel 4: Final Intersection Mask (in Green)
#             ax4.imshow(target_rgb)
#             if intersection_mask is not None:
#                 target_overlay_4 = target_rgb.copy()
#                 roi_int = target_overlay_4[intersection_mask > 0]
#                 blended_int = (roi_int * 0.5 + np.array([0, 255, 0], dtype=np.uint8) * 0.5).astype(np.uint8)
#                 target_overlay_4[intersection_mask > 0] = blended_int
#                 ax4.imshow(target_overlay_4)
#             ax4.set_title("4. Final High-Confidence Intersection"); ax4.axis('off')
            
#             plt.tight_layout(rect=[0, 0, 1, 0.93])
#             plt.show()

# print(f"\nAll {len(target_components)} specific component analyses are complete.")

Comparison of add and not add RANSAC & 3D mesh (specific components)

In [ ]:
# --- Pre-processing to Find Best Views ---
print("--- Pre-processing: Finding the best view for each component ---")
component_area_map = {}
normalized_name_to_id = {os.path.splitext(os.path.basename(img.name))[0]: img_id for img_id, img in images.items()}
for normalized_name, annotations in image_name_to_anns.items():
    if normalized_name in normalized_name_to_id:
        image_id = normalized_name_to_id[normalized_name]
        for ann in annotations:
            component_name = cat_id_to_name[ann['category_id']]
            if component_name not in component_area_map: component_area_map[component_name] = []
            mask = mask_utils.decode(ann['segmentation']); area = np.sum(mask > 0)
            component_area_map[component_name].append((area, image_id))
for component_name in component_area_map:
    component_area_map[component_name].sort(key=lambda x: x[0], reverse=True)
print("Pre-processing complete.")

# # --- Main Execution Block for Specific Components ---

# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running Analysis for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")
    
#     source_id, target_id = None, None
    
#     # Get the best view (largest area) for the specified component
#     best_view_list = component_area_map.get(component_name, [])
#     if best_view_list:
#         source_id = best_view_list[0][1]
#         target_id = random.choice([img_id for img_id in valid_image_ids if img_id != source_id])
    
#     if source_id:
#         print(f"--- Starting Head-to-Head Comparison ---\nComponent: '{component_name}'\nSource: {images[source_id].name}\nTarget: {images[target_id].name}")
#         source_mask, _ = get_component_mask(images[source_id].name, component_name, image_name_to_anns, cat_id_to_name)
        
#         if source_mask is not None:
#             mask_v1 = transfer_mask_v1(source_id, target_id, source_mask)
#             mask_v2 = transfer_mask_v2(source_id, target_id, source_mask, component_name)
            
#             source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
#             target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
            
#             target_gt_mask, _ = get_component_mask(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name)
            
#             # --- Conditional Plotting Logic ---
#             if target_gt_mask is not None:
#                 # IoU Path (4-Panel Plot)
#                 iou_v1 = calculate_iou(mask_v1, target_gt_mask)
#                 iou_v2 = calculate_iou(mask_v2, target_gt_mask)
#                 print(f"  - IoU for 'Robust': {iou_v1:.4f} | IoU for 'Advanced': {iou_v2:.4f}")

#                 fig, axes = plt.subplots(1, 4, figsize=(40, 10))
#                 fig.suptitle(f"IoU Comparison for '{component_name}'", fontsize=24, y=1.0)
#                 (ax1, ax2, ax3, ax4) = axes

#                 ax1.imshow(source_rgb); ax1.imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.5); ax1.set_title("1. Source Mask"); ax1.axis('off')
#                 ax2.imshow(target_rgb); overlay_v1 = target_rgb.copy(); roi_v1 = overlay_v1[mask_v1 > 0]; blended_v1 = (roi_v1*0.5+np.array([0,0,255],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v1[mask_v1 > 0] = blended_v1; ax2.imshow(overlay_v1); ax2.set_title(f"2. 'v1' (IoU: {iou_v1:.4f})"); ax2.axis('off')
#                 ax3.imshow(target_rgb); overlay_v2 = target_rgb.copy(); roi_v2 = overlay_v2[mask_v2 > 0]; blended_v2 = (roi_v2*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v2[mask_v2 > 0] = blended_v2; ax3.imshow(overlay_v2); ax3.set_title(f"3. 'v2' (IoU: {iou_v2:.4f})"); ax3.axis('off')
#                 gt_overlay = target_rgb.copy(); roi_gt = gt_overlay[target_gt_mask > 0]; blended_gt = (roi_gt * 0.5 + np.array([0, 255, 0], dtype=np.uint8) * 0.5).astype(np.uint8); gt_overlay[target_gt_mask > 0] = blended_gt; ax4.imshow(gt_overlay); ax4.set_title("4. Ground Truth"); ax4.axis('off')

#             else:
#                 # Spread Rate Path (3-Panel Plot)
#                 target_vehicle_mask = cv2.imread(image_id_to_mask_path.get(target_id, ''), cv2.IMREAD_GRAYSCALE)
#                 spread_v1 = calculate_projection_spread(mask_v1, target_vehicle_mask)
#                 spread_v2 = calculate_projection_spread(mask_v2, target_vehicle_mask)
#                 print(f"  - Spread for 'Robust': {spread_v1:.2%} | Spread for 'Advanced': {spread_v2:.2%}")

#                 fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
#                 fig.suptitle(f"Spread Rate Comparison for '{component_name}'", fontsize=24, y=1.0)
                
#                 ax1.imshow(source_rgb); ax1.imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.5); ax1.set_title("1. Source Mask"); ax1.axis('off')
#                 ax2.imshow(target_rgb); overlay_v1 = target_rgb.copy(); roi_v1 = overlay_v1[mask_v1 > 0]; blended_v1 = (roi_v1*0.5+np.array([0,0,255],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v1[mask_v1 > 0] = blended_v1; ax2.imshow(overlay_v1); ax2.set_title(f"2. 'v1' (Spread: {spread_v1:.2%})"); ax2.axis('off')
#                 ax3.imshow(target_rgb); overlay_v2 = target_rgb.copy(); roi_v2 = overlay_v2[mask_v2 > 0]; blended_v2 = (roi_v2*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v2[mask_v2 > 0] = blended_v2; ax3.imshow(overlay_v2); ax3.set_title(f"3. 'v2' (Spread: {spread_v2:.2%})"); ax3.axis('off')

#             plt.tight_layout(rect=[0, 0, 1, 0.95])
#             plt.show()
#     else:
#         print(f"Could not find a valid source image for component '{component_name}'. It may not exist in the dataset.")

# print(f"\nAll {len(target_components)} specific component analyses are complete.")

## Phase5: Version 3 dense mask transfer (Display part that visible)

Comparison of transfer_mask_v2 and transfer_mask_v3

In [ ]:
# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running Ultimate Comparison for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")
    
#     source_id, target_id = None, None
#     best_view_list = component_area_map.get(component_name, [])
#     if best_view_list:
#         source_id = best_view_list[0][1]
#         target_id = random.choice([img_id for img_id in valid_image_ids if img_id != source_id])
    
#     if source_id:
#         print(f"--- Component: '{component_name}' | Source: {images[source_id].name} | Target: {images[target_id].name} ---")
        
#         source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
#         source_h, source_w, _ = source_rgb.shape
#         source_mask, _ = get_component_mask_robust(images[source_id].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)
        
#         if source_mask is not None:
#             # --- UPDATED: Run both transfer functions with all required arguments ---
#             mask_v_advanced = transfer_mask_v2(
#                 source_id, target_id, source_mask, component_name,
#                 images, cameras, image_id_to_rgb_path, image_id_to_depth_path
#             )
#             mask_v_ultimate = transfer_mask_v3(
#                 source_id, target_id, source_mask, component_name,
#                 images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path
#             )
            
#             target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)

#             fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 10))
#             fig.suptitle(f"Final Refinement Comparison for '{component_name}'", fontsize=24, y=1.0)

#             ax1.imshow(source_rgb); ax1.imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.5); ax1.set_title("1. Source Mask"); ax1.axis('off')
#             ax2.imshow(target_rgb); overlay_v2 = target_rgb.copy(); roi_v2 = overlay_v2[mask_v_advanced > 0]; blended_v2 = (roi_v2*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v2[mask_v_advanced > 0] = blended_v2; ax2.imshow(overlay_v2)
#             ax2.set_title("2. v2 Method (Self-Occlusion Only)", fontsize=18); ax2.axis('off')
#             ax3.imshow(target_rgb); overlay_v3 = target_rgb.copy(); roi_v3 = overlay_v3[mask_v_ultimate > 0]; blended_v3 = (roi_v3*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v3[mask_v_ultimate > 0] = blended_v3; ax3.imshow(overlay_v3)
#             ax3.set_title("3. v3 Method (Full Check & Clip)", fontsize=18); ax3.axis('off')
            
#             plt.tight_layout(rect=[0, 0, 1, 0.95])
#             plt.show()
#     else:
#         print(f"Could not find a valid source image for component '{component_name}'.")

# print(f"\nAll {len(target_components)} specific component analyses are complete.")

## Phase6: Different Point of View Combinations (Cross-Sections of 3 source and 5 source)

In [ ]:
# # --- Run 1 example with 3 source views from your specific list ---
# run_multi_view(
#     num_source_views=3,
#     example_num=1,
#     specific_components=target_components,
#     # Pass in all the necessary data from your notebook
#     component_to_views_map=component_to_views_map,
#     images=images,
#     image_name_to_anns=image_name_to_anns,
#     cat_id_to_name=cat_id_to_name,
#     image_id_to_rgb_path=image_id_to_rgb_path,
#     image_id_to_depth_path=image_id_to_depth_path,
#     image_id_to_mask_path=image_id_to_mask_path,
#     cameras=cameras
# )

# # --- Run 1 example with 5 source views from your specific list ---
# run_multi_view(
#     num_source_views=5,
#     example_num=1,
#     specific_components=target_components,
#     # Pass in all the necessary data from your notebook
#     component_to_views_map=component_to_views_map,
#     images=images,
#     image_name_to_anns=image_name_to_anns,
#     cat_id_to_name=cat_id_to_name,
#     image_id_to_rgb_path=image_id_to_rgb_path,
#     image_id_to_depth_path=image_id_to_depth_path,
#     image_id_to_mask_path=image_id_to_mask_path,
#     cameras=cameras
# )

# print("\nAll specific multi-view consistency checks are complete.")

## Phase7: V3 (4 corners source image + Union sections)

In [ ]:
# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running 4-Corner Analysis for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")
    
#     source_ids = find_corner_views(component_name, component_to_views_map, images)
#     if len(source_ids) < 2:
#         print(f"Not enough diverse corner views found for '{component_name}'. Skipping.")
#         continue
        
#     target_id = random.choice([img_id for img_id in valid_image_ids if img_id not in source_ids])
#     print(f"\nProjecting {len(source_ids)} source views to Target: {images[target_id].name}")

#     projected_masks, source_masks_for_viz, source_rgbs_for_viz = [], [], []
#     for sid in source_ids:
#         source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
#         if source_mask is not None:
#             transferred_mask = transfer_mask_v3(sid, target_id, source_mask, component_name)
#             if transferred_mask is not None:
#                 projected_masks.append(transferred_mask)
#                 source_masks_for_viz.append(source_mask)
#                 source_rgbs_for_viz.append(cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB))

#     if len(projected_masks) < 2:
#         print("Failed to get enough successful projections. Skipping.")
#         continue

#     # --- Combine with Union and Intersection ---
#     target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
#     union_mask = np.zeros((target_h, target_w), dtype=np.uint8)
#     intersection_mask = np.full((target_h, target_w), 255, dtype=np.uint8)
#     for mask in projected_masks:
#         union_mask = cv2.bitwise_or(union_mask, mask)
#         intersection_mask = cv2.bitwise_and(intersection_mask, mask)
        
#     # --- Perform Dual Evaluation ---
#     target_gt_mask, _ = get_component_mask(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name)
#     target_vehicle_mask = cv2.imread(image_id_to_mask_path.get(target_id, ''), cv2.IMREAD_GRAYSCALE)
    
#     # --- Comprehensive Visualization ---
#     num_sources = len(source_masks_for_viz)
#     # Determine number of columns needed (max of sources or results)
#     num_cols = max(num_sources, 3 if target_gt_mask is not None else 2)
#     fig, axes = plt.subplots(3, num_cols, figsize=(10 * num_cols, 30))
#     fig.suptitle(f"Complete 4-Corner Analysis for '{component_name}'", fontsize=30, y=0.95)

#     target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)

#     # --- Row 1: Source Masks ---
#     for j in range(num_sources):
#         axes[0, j].imshow(source_rgbs_for_viz[j]); axes[0, j].imshow(np.ma.masked_where(source_masks_for_viz[j]==0, source_masks_for_viz[j]), cmap='cool', alpha=0.6)
#         axes[0, j].set_title(f"Source {chr(65+j)}"); axes[0, j].axis('off')
#     for j in range(num_sources, num_cols): axes[0, j].axis('off') # Hide unused axes

#     # --- Row 2: Individual Projected Masks ---
#     for j in range(num_sources):
#         axes[1, j].imshow(target_rgb); axes[1, j].imshow(np.ma.masked_where(projected_masks[j]==0, projected_masks[j]), cmap='plasma', alpha=0.6)
#         axes[1, j].set_title(f"Projection from Source {chr(65+j)}"); axes[1, j].axis('off')
#     for j in range(num_sources, num_cols): axes[1, j].axis('off')

#     # --- Row 3: Final Combined Results ---
#     # Union Result
#     axes[2, 0].imshow(target_rgb); overlay_u = target_rgb.copy(); roi_u = overlay_u[union_mask > 0]; blended_u = (roi_u*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_u[union_mask > 0] = blended_u; axes[2, 0].imshow(overlay_u)
    
#     # Intersection Result
#     axes[2, 1].imshow(target_rgb); overlay_i = target_rgb.copy(); roi_i = overlay_i[intersection_mask > 0]; blended_i = (roi_i*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_i[intersection_mask > 0] = blended_i; axes[2, 1].imshow(overlay_i)

#     # Set titles with appropriate scores
#     if target_gt_mask is not None:
#         iou_union = calculate_iou(union_mask, target_gt_mask)
#         iou_intersection = calculate_iou(intersection_mask, target_gt_mask)
#         axes[2, 0].set_title(f"Final Union (IoU: {iou_union:.4f})", fontsize=18)
#         axes[2, 1].set_title(f"Final Intersection (IoU: {iou_intersection:.4f})", fontsize=18)
        
#         # Ground Truth Result
#         axes[2, 2].imshow(target_rgb); overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes[2, 2].imshow(overlay_gt)
#         axes[2, 2].set_title("Ground Truth", fontsize=18); axes[2, 2].axis('off')
#         for j in range(3, num_cols): axes[2, j].axis('off') # Hide remaining axes

#     else:
#         spread_union = calculate_projection_spread(union_mask, target_vehicle_mask)
#         spread_intersection = calculate_projection_spread(intersection_mask, target_vehicle_mask)
#         axes[2, 0].set_title(f"Final Union (Spread: {spread_union:.2%})", fontsize=18)
#         axes[2, 1].set_title(f"Final Intersection (Spread: {spread_intersection:.2%})", fontsize=18)
#         for j in range(2, num_cols): axes[2, j].axis('off')

#     axes[2, 0].axis('off')
#     axes[2, 1].axis('off')

#     plt.tight_layout(rect=[0, 0, 1, 0.93])
#     plt.show()

# print(f"\nAll {len(target_components)} specific component analyses are complete.")

## Phase8: V4 dense mask transfer (Remove edge pixels)

In [ ]:
# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running Ultimate 4-Corner Analysis for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")
    
#     source_ids = find_corner_views(component_name, component_to_views_map, images)
#     if len(source_ids) < 2:
#         print(f"Not enough diverse corner views found. Skipping.")
#         continue
        
#     target_id = random.choice([img_id for img_id in valid_image_ids if img_id not in source_ids])
#     print(f"\nProjecting {len(source_ids)} source views to Target: {images[target_id].name}")

#     # --- Run BOTH projection methods for all corners ---
#     projected_masks_v3, projected_masks_v4 = [], []
#     source_masks_for_viz, source_rgbs_for_viz = [], []
    
#     for sid in source_ids:
#         source_rgb_for_viz = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB)
#         source_h, source_w, _ = source_rgb_for_viz.shape
#         source_mask, _ = get_component_mask_robust(images[sid].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)

#         if source_mask is not None:
#             source_masks_for_viz.append(source_mask)
#             source_rgbs_for_viz.append(source_rgb_for_viz)
            
#             # V3
#             transfer_v3 = transfer_mask_v3(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
#             if transfer_v3 is not None: projected_masks_v3.append(transfer_v3)
#             # V4
#             transfer_v4 = transfer_mask_v4(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
#             if transfer_v4 is not None: projected_masks_v4.append(transfer_v4)

#     if not projected_masks_v3 or not projected_masks_v4:
#         print("Failed to get enough successful projections. Skipping.")
#         continue

#     # --- Combine with Union and Calculate IoU for both ---
#     target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
#     union_mask_v3 = np.zeros((target_h, target_w), dtype=np.uint8)
#     union_mask_v4 = np.zeros((target_h, target_w), dtype=np.uint8)
#     for mask in projected_masks_v3: union_mask_v3 = cv2.bitwise_or(union_mask_v3, mask)
#     for mask in projected_masks_v4: union_mask_v4 = cv2.bitwise_or(union_mask_v4, mask)
        
#     target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
#     target_h_gt, target_w_gt, _ = target_rgb.shape
#     target_gt_mask, _ = get_component_mask_robust(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name, target_h_gt, target_w_gt)
    
#     iou_v3, iou_v4 = 0.0, 0.0
#     if target_gt_mask is not None:
#         iou_v3 = calculate_iou(union_mask_v3, target_gt_mask)
#         iou_v4 = calculate_iou(union_mask_v4, target_gt_mask)
#         print(f"\n>>> IoU for V3 Union (Full Mask): {iou_v3:.4f} <<<")
#         print(f">>> IoU for V4 Union (Edge-Removed): {iou_v4:.4f} <<<")
#     else:
#         print("\nNo ground truth found for target view. Skipping IoU calculation.")

#     # --- Comprehensive Visualization ---
#     num_sources = len(source_masks_for_viz)
#     fig, axes = plt.subplots(3, num_sources, figsize=(10 * num_sources, 30), sharex=True, sharey=True)
#     fig.suptitle(f"Ultimate 4-Corner Analysis for '{component_name}'", fontsize=30, y=0.95)

#     # Row 1: Source Masks
#     for j in range(num_sources):
#         axes[0, j].imshow(source_rgbs_for_viz[j]); axes[0, j].imshow(np.ma.masked_where(source_masks_for_viz[j]==0, source_masks_for_viz[j]), cmap='cool', alpha=0.6)
#         axes[0, j].set_title(f"Source {chr(65+j)}"); axes[0, j].axis('off')
#     # Hide unused axes
#     for j in range(num_sources, 4):
#         if j < axes.shape[1]: axes[0,j].axis('off')

#     # Row 2: Individual Projections (V3)
#     for j in range(len(projected_masks_v3)):
#         axes[1, j].imshow(target_rgb); axes[1, j].imshow(np.ma.masked_where(projected_masks_v3[j]==0, projected_masks_v3[j]), cmap='Blues', alpha=0.6)
#         axes[1, j].set_title(f"V3 Projection from Source {chr(65+j)}"); axes[1, j].axis('off')
#     for j in range(len(projected_masks_v3), 4):
#         if j < axes.shape[1]: axes[1,j].axis('off')
        
#     # Row 3: Individual Projections (V4)
#     for j in range(len(projected_masks_v4)):
#         axes[2, j].imshow(target_rgb); axes[2, j].imshow(np.ma.masked_where(projected_masks_v4[j]==0, projected_masks_v4[j]), cmap='Reds', alpha=0.6)
#         axes[2, j].set_title(f"V4 Projection from Source {chr(65+j)}"); axes[2, j].axis('off')
#     for j in range(len(projected_masks_v4), 4):
#         if j < axes.shape[1]: axes[2,j].axis('off')
    
#     plt.tight_layout(rect=[0, 0, 1, 0.92])
#     plt.show()

#     # --- Final Comparison Plot ---
#     fig_final, axes_final = plt.subplots(1, 3, figsize=(30, 10))
#     fig_final.suptitle(f"Final Combined Results for '{component_name}'", fontsize=24, y=1.0)
    
#     # Final V3 Union
#     axes_final[0].imshow(target_rgb); overlay_v3 = target_rgb.copy(); roi_v3 = overlay_v3[union_mask_v3 > 0]; blended_v3 = (roi_v3*0.5+np.array([0,0,255],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v3[union_mask_v3 > 0] = blended_v3; axes_final[0].imshow(overlay_v3)
#     axes_final[0].set_title(f"V3 Union Result (IoU: {iou_v3:.4f})", fontsize=18); axes_final[0].axis('off')

#     # Final V4 Union
#     axes_final[1].imshow(target_rgb); overlay_v4 = target_rgb.copy(); roi_v4 = overlay_v4[union_mask_v4 > 0]; blended_v4 = (roi_v4*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v4[union_mask_v4 > 0] = blended_v4; axes_final[1].imshow(overlay_v4)
#     axes_final[1].set_title(f"V4 Union Result (IoU: {iou_v4:.4f})", fontsize=18); axes_final[1].axis('off')

#     # Ground Truth
#     axes_final[2].imshow(target_rgb)
#     if target_gt_mask is not None:
#         overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes_final[2].imshow(overlay_gt)
#     axes_final[2].set_title("Ground Truth", fontsize=18); axes_final[2].axis('off')
    
#     plt.tight_layout(rect=[0, 0, 1, 0.93])
#     plt.show()

# print(f"\nAll {len(target_components)} specific component analyses are complete.")

## Phase9: Each Component 10 combination Average IOU (140 mins)

In [ ]:
def run_randomized_corner_analysis(method_function, component_name, num_combinations=10):
    """
    Runs the analysis for a single component and returns the list of IoU scores.
    """
    print(f"  - Analyzing '{component_name}' with {method_function.__name__}...")
    source_ids = find_corner_views(component_name, component_to_views_map, images)
    if len(source_ids) < 2:
        print(f"    - Progress: Not enough diverse corner views found. Skipping.")
        return None
    valid_targets = []
    potential_target_ids = [img_id for img_id in valid_image_ids if img_id not in source_ids]
    for target_id in potential_target_ids:
        target_gt_mask, _ = get_component_mask(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name)
        if target_gt_mask is not None:
            valid_targets.append(target_id)
    if not valid_targets:
        print(f"    - Progress: No valid target images with ground truth found.")
        return None
    targets_to_test = random.sample(valid_targets, min(num_combinations, len(valid_targets)))
    print(f"    - Progress: Found {len(valid_targets)} valid targets. Testing on {len(targets_to_test)} random combinations...")
    iou_scores = []
    for idx, target_id in enumerate(targets_to_test):
        print(f"      ({idx+1}/{len(targets_to_test)}) Testing Target: {images[target_id].name}...", end='', flush=True)
        projected_masks = []
        for sid in source_ids:
            source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
            if source_mask is not None:
                transferred_mask = method_function(sid, target_id, source_mask, component_name)
                if transferred_mask is not None:
                    projected_masks.append(transferred_mask)
        if len(projected_masks) < len(source_ids):
            print(" Projection failed.")
            continue
        target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
        union_mask = np.zeros((target_h, target_w), dtype=np.uint8)
        for mask in projected_masks:
            union_mask = cv2.bitwise_or(union_mask, mask)
        iou_score = calculate_iou(union_mask, target_gt_mask)
        iou_scores.append(iou_score)
        print(f" IoU: {iou_score:.4f}")
    return iou_scores

In [ ]:
results_file_path = 'iou_results.json'

if os.path.exists(results_file_path):
    print(f"Loading existing results from '{results_file_path}'...")
    with open(results_file_path, 'r') as f:
        results = json.load(f)
else:
    print("No existing results file found. Starting a new analysis.")
    results = {"v3": {}, "v4": {}}

# --- Run Analysis for V3 ---
print(f"\n{'='*25}\n--- Checking for missing V3 analyses ---\n{'='*25}")
for component_name in target_components:
    if component_name not in results.get("v3", {}):
        scores = run_randomized_corner_analysis(transfer_mask_v3, component_name, num_combinations=10)
        if scores:
            if "v3" not in results: results["v3"] = {}
            results["v3"][component_name] = scores
            with open(results_file_path, 'w') as f:
                json.dump(results, f, indent=4)
            print(f"    - Saved results for '{component_name}' to file.")
    else:
        print(f"  - Analysis for '{component_name}' (V3) already complete. Skipping.")

# --- Run Analysis for V4 ---
print(f"\n{'='*25}\n--- Checking for missing V4 analyses ---\n{'='*25}")
for component_name in target_components:
    if component_name not in results.get("v4", {}):
        scores = run_randomized_corner_analysis(transfer_mask_v4, component_name, num_combinations=10)
        if scores:
            if "v4" not in results: results["v4"] = {}
            results["v4"][component_name] = scores
            with open(results_file_path, 'w') as f:
                json.dump(results, f, indent=4)
            print(f"    - Saved results for '{component_name}' to file.")
    else:
        print(f"  - Analysis for '{component_name}' (V4) already complete. Skipping.")

# --- 5. Final Consolidated Report ---
print("\n\n" + "="*40)
print("      FINAL QUANTITATIVE REPORT")
print("="*40)
report_lines = []
for method_name, method_results in results.items():
    report_lines.append(f"\n--- Method: {method_name} ---")
    if not method_results:
        report_lines.append("No valid results were generated for this method.")
    else:
        all_scores_for_method = []
        sorted_components = sorted(method_results.keys())
        for component_name in sorted_components:
            scores = method_results[component_name]
            all_scores_for_method.extend(scores)
            average_iou = np.mean(scores)
            report_lines.append(f"\n- Component: '{component_name}'")
            report_lines.append(f"  - Individual IoU Scores: {', '.join([f'{s:.4f}' for s in scores])}")
            report_lines.append(f"  >>> Average IoU ({len(scores)} combinations): {average_iou:.4f}")
        if all_scores_for_method:
            overall_average = np.mean(all_scores_for_method)
            report_lines.append("\n" + "-"*35)
            report_lines.append(f">>> OVERALL AVERAGE IoU for {method_name.upper()}: {overall_average:.4f} <<<")
            report_lines.append("-"*35)
final_report_string = "\n".join(report_lines)
with open('final_iou_report.txt', 'w') as f_out:
    f_out.write(final_report_string)
print(final_report_string)
print("\n\nAll analyses are complete. Full report saved to 'final_iou_report.txt'.")

## Phase 10: V3 Dense mask trasnsfer (Faster Verison) + 4 corner analysis

Comparing v3 and v3_fast

In [ ]:
# for i, component_name in enumerate(target_components):
#     print(f"\n{'='*20} Running Speed vs. Accuracy for '{component_name}' ({i+1} of {len(target_components)}) {'='*20}\n")
    
#     source_ids = find_corner_views(component_name, component_to_views_map, images)
#     if len(source_ids) < 2:
#         print(f"Not enough diverse corner views found. Skipping.")
#         continue
        
#     # *** THIS IS THE CORRECTED LOGIC FOR FINDING VALID TARGETS ***
#     valid_targets = []
#     for tid in valid_image_ids:
#         if tid not in source_ids:
#             # Look up the correct camera to get height and width
#             cam_id = images[tid].camera_id
#             h, w = cameras[cam_id].height, cameras[cam_id].width
#             # Now check for the mask
#             if get_component_mask_robust(images[tid].name, component_name, image_name_to_anns, cat_id_to_name, h, w)[0] is not None:
#                 valid_targets.append(tid)

#     if not valid_targets:
#         print("Could not find a valid target with a ground truth mask. Skipping.")
#         continue
#     target_id = random.choice(valid_targets)
    
#     print(f"Projecting {len(source_ids)} source views to Target: {images[target_id].name}")

#     source_masks_for_viz = []
#     source_rgbs_for_viz = []

#     # --- Time and run V3 (Original) ---
#     start_time_v3 = time.time()
#     projected_masks_v3 = []
#     for sid in source_ids:
#         source_cam_id = images[sid].camera_id
#         source_h, source_w = cameras[source_cam_id].height, cameras[source_cam_id].width
#         source_mask, _ = get_component_mask_robust(images[sid].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)
        
#         if source_mask is not None:
#             source_masks_for_viz.append(source_mask)
#             source_rgbs_for_viz.append(cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB))

#             transferred_mask = transfer_mask_v3(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
#             if transferred_mask is not None:
#                 projected_masks_v3.append(transferred_mask)
#     end_time_v3 = time.time()
#     time_v3 = end_time_v3 - start_time_v3

#     # --- Time and run V3-Fast (Optimized) ---
#     start_time_v3_fast = time.time()
#     projected_masks_v3_fast = []
#     for idx, sid in enumerate(source_ids):
#         source_mask = source_masks_for_viz[idx]
#         transferred_mask = transfer_mask_v3_fast(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
#         if transferred_mask is not None:
#             projected_masks_v3_fast.append(transferred_mask)
#     end_time_v3_fast = time.time()
#     time_v3_fast = end_time_v3_fast - start_time_v3_fast

#     # --- Combine results and calculate IoU ---
#     target_cam_id = images[target_id].camera_id
#     target_h, target_w = cameras[target_cam_id].height, cameras[target_cam_id].width
#     union_mask_v3 = np.zeros((target_h, target_w), dtype=np.uint8)
#     union_mask_v3_fast = np.zeros((target_h, target_w), dtype=np.uint8)
#     for mask in projected_masks_v3: union_mask_v3 = cv2.bitwise_or(union_mask_v3, mask)
#     for mask in projected_masks_v3_fast: union_mask_v3_fast = cv2.bitwise_or(union_mask_v3_fast, mask)
        
#     target_gt_mask, _ = get_component_mask_robust(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name, target_h, target_w)
#     iou_v3 = calculate_iou(union_mask_v3, target_gt_mask)
#     iou_v3_fast = calculate_iou(union_mask_v3_fast, target_gt_mask)
    
#     print(f"\n--- Results for '{component_name}' ---")
#     print(f"V3 (Original):   Time: {time_v3:.2f}s | IoU: {iou_v3:.4f}")
#     print(f"V3-Fast (Optimized): Time: {time_v3_fast:.2f}s | IoU: {iou_v3_fast:.4f}")
#     speedup = time_v3 / time_v3_fast if time_v3_fast > 0 else float('inf')
#     print(f">>> Speedup: {speedup:.2f}x <<<")

#     # --- Comprehensive Visualization ---
#     num_sources = len(source_masks_for_viz)
#     fig, axes = plt.subplots(3, num_sources, figsize=(10 * num_sources, 30), sharex=True, sharey=True)
#     fig.suptitle(f"Detailed V3 vs. V3-Fast Analysis for '{component_name}'", fontsize=30, y=0.95)
#     target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)

#     # Row 1: Source Masks
#     for j in range(num_sources):
#         axes[0, j].imshow(source_rgbs_for_viz[j]); axes[0, j].imshow(np.ma.masked_where(source_masks_for_viz[j]==0, source_masks_for_viz[j]), cmap='cool', alpha=0.6)
#         axes[0, j].set_title(f"Source {chr(65+j)}"); axes[0, j].axis('off')
#     for j in range(num_sources, 4): 
#         if j < axes.shape[1]: axes[0, j].axis('off')

#     # Row 2: V3 Individual Projections
#     for j in range(len(projected_masks_v3)):
#         axes[1, j].imshow(target_rgb); axes[1, j].imshow(np.ma.masked_where(projected_masks_v3[j]==0, projected_masks_v3[j]), cmap='Blues', alpha=0.6)
#         axes[1, j].set_title(f"V3 Projection from {chr(65+j)}"); axes[1, j].axis('off')
#     for j in range(len(projected_masks_v3), 4): 
#         if j < axes.shape[1]: axes[1, j].axis('off')
        
#     # Row 3: V3-Fast Individual Projections
#     for j in range(len(projected_masks_v3_fast)):
#         axes[2, j].imshow(target_rgb); axes[2, j].imshow(np.ma.masked_where(projected_masks_v3_fast[j]==0, projected_masks_v3_fast[j]), cmap='Reds', alpha=0.6)
#         axes[2, j].set_title(f"V3-Fast Projection from {chr(65+j)}"); axes[2, j].axis('off')
#     for j in range(len(projected_masks_v3_fast), 4): 
#         if j < axes.shape[1]: axes[2, j].axis('off')
    
#     plt.tight_layout(rect=[0, 0, 1, 0.92])
#     plt.show()
    
#     # --- Final Comparison Plot ---
#     fig_final, axes_final = plt.subplots(1, 3, figsize=(30, 10), sharex=True, sharey=True)
#     fig_final.suptitle(f"Final Combined Results for '{component_name}'", fontsize=24)
    
#     # Final V3 Union
#     axes_final[0].imshow(target_rgb); overlay_v3 = target_rgb.copy(); roi_v3 = overlay_v3[union_mask_v3 > 0]; blended_v3 = (roi_v3*0.5+np.array([0,0,255],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v3[union_mask_v3 > 0] = blended_v3; axes_final[0].imshow(overlay_v3)
#     axes_final[0].set_title(f"V3 Union (Original)\nTime: {time_v3:.2f}s | IoU: {iou_v3:.4f}", fontsize=18); axes_final[0].axis('off')
    
#     # Final V3-Fast Union
#     axes_final[1].imshow(target_rgb); overlay_v3_fast = target_rgb.copy(); roi_v3_fast = overlay_v3_fast[union_mask_v3_fast > 0]; blended_v3_fast = (roi_v3_fast*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_v3_fast[union_mask_v3_fast > 0] = blended_v3_fast; axes_final[1].imshow(overlay_v3_fast)
#     axes_final[1].set_title(f"V3-Fast Union (Optimized)\nTime: {time_v3_fast:.2f}s | IoU: {iou_v3_fast:.4f}", fontsize=18); axes_final[1].axis('off')
    
#     # Ground Truth
#     axes_final[2].imshow(target_rgb); overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes_final[2].imshow(overlay_gt)
#     axes_final[2].set_title("Ground Truth", fontsize=18); axes_final[2].axis('off')

#     plt.tight_layout(rect=[0, 0, 1, 0.93])
#     plt.show()

# print(f"\nAll {len(target_components)} specific component analyses are complete.")

4 corner analysis for v3_fast with different subsample steps (3-20)

In [ ]:
results_file_path = 'iou_subsample_results.json'
subsample_steps_to_test = [3, 4, 5, 6, 7, 8, 9, 10, 20]
num_combinations = 20

if os.path.exists(results_file_path):
    print(f"Loading existing results from '{results_file_path}'...")
    with open(results_file_path, 'r') as f:
        results = json.load(f)
else:
    results = {}

# --- ANALYSIS LOOP ---
for component_name in target_components:
    print(f"\n{'='*25}\n--- Preparing Analysis for '{component_name}' ---\n{'='*25}")
    
    if component_name not in results:
        results[component_name] = {}
        
    # Step 1: Find a fixed set of target images for this component
    source_ids_for_comp = find_corner_views(component_name, component_to_views_map, images)
    if len(source_ids_for_comp) < 2:
        print(f"Not enough diverse corner views found for '{component_name}'. Skipping.")
        continue
    
    # *** THIS IS THE CORRECTED LOGIC FOR FINDING VALID TARGETS ***
    valid_targets = []
    for tid in valid_image_ids:
        if tid not in source_ids_for_comp:
            # Look up the correct camera to get height and width
            cam_id = images[tid].camera_id
            h, w = cameras[cam_id].height, cameras[cam_id].width
            # Now check for the mask
            if get_component_mask_robust(images[tid].name, component_name, image_name_to_anns, cat_id_to_name, h, w)[0] is not None:
                valid_targets.append(tid)

    if not valid_targets:
        print(f"No valid target images with ground truth found for '{component_name}'. Skipping.")
        continue
        
    targets_to_test = random.sample(valid_targets, min(num_combinations, len(valid_targets)))
    print(f"Selected {len(targets_to_test)} random target images for direct comparison.")

    # Step 2: Loop through subsample steps and test on the SAME target set
    for step in subsample_steps_to_test:
        step_str = str(step)
        if step_str not in results[component_name]:
            scores, total_time = run_subsample_analysis_on_fixed_targets(
                component_name,
                subsample_step=step,
                targets_to_test=targets_to_test,
                component_to_views_map=component_to_views_map,
                images=images,
                image_name_to_anns=image_name_to_anns,
                cat_id_to_name=cat_id_to_name,
                image_id_to_rgb_path=image_id_to_rgb_path,
                image_id_to_depth_path=image_id_to_depth_path,
                image_id_to_mask_path=image_id_to_mask_path,
                cameras=cameras
            )
            if scores:
                avg_iou = np.mean(scores)
                results[component_name][step_str] = {"avg_iou": avg_iou, "time": total_time, "individual_scores": scores}
                with open(results_file_path, 'w') as f:
                    json.dump(results, f, indent=4)
        else:
            print(f"  - Analysis for subsample_step={step} already complete. Skipping.")

# --- 4. Final Consolidated Report ---
print("\n\n" + "="*50)
print("      FINAL SUBSAMPLING SPEED VS. ACCURACY REPORT")
print("="*50)
report_lines = []
for component_name, component_results in results.items():
    report_lines.append(f"\n--- Component: '{component_name}' ---")
    # Sort the results by step size numerically
    for step_str, data in sorted(component_results.items(), key=lambda item: int(item[0])):
        report_lines.append(f"  - Subsample Step: {step_str}x{step_str} | Avg IoU: {data['avg_iou']:.4f} | Total Time: {data['time']:.2f}s")
final_report_string = "\n".join(report_lines)
with open('final_subsample_report.txt', 'w') as f_out:
    f_out.write(final_report_string)
print(final_report_string)
print(f"\n\nAll subsampling analyses are complete. Full report saved to 'final_subsample_report.txt'.")

## Phase 11: Testing on 'BUMPER' with subsample steps 7-9 and Visualisation

In [ ]:
def run_component_analysis(component_name, subsample_step, low_iou_threshold=0.65):
    """
    Runs the 4-corner analysis for a specific component, saves low-IoU plots,
    and returns IoU scores and total execution time.
    """
    print(f"\n--- Running Full Analysis for '{component_name}' with subsample_step={subsample_step} ---")
    
    source_ids = find_corner_views(component_name, component_to_views_map, images)
    if len(source_ids) < 2:
        print(f"Not enough diverse corner views found. Skipping.")
        return [], 0.0

    valid_targets = [tid for tid in valid_image_ids if tid not in source_ids and get_component_mask(images[tid].name, component_name, image_name_to_anns, cat_id_to_name)[0] is not None]
    if not valid_targets:
        print(f"No valid target images with ground truth found.")
        return [], 0.0

    print(f"Found {len(valid_targets)} valid targets. Testing on all of them...")
    
    iou_scores = []
    
    # --- Nested output folder structure ---
    main_output_folder = "low_iou_failures"
    sanitized_component_name = component_name.replace('/', '_')
    output_folder = os.path.join(main_output_folder, sanitized_component_name, f"step_{subsample_step}")
    os.makedirs(output_folder, exist_ok=True)
    
    start_time = time.time()
    
    for idx, target_id in enumerate(valid_targets):
        target_gt_mask, _ = get_component_mask(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name)
        
        print(f"  ({idx+1}/{len(valid_targets)}) Testing Target: {images[target_id].name}...", end='', flush=True)
        
        projected_masks = []
        source_masks_for_viz, source_rgbs_for_viz = [], []
        
        for sid in source_ids:
            source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
            if source_mask is not None:
                source_masks_for_viz.append(source_mask)
                source_rgbs_for_viz.append(cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB))
                transferred_mask = transfer_mask_v3_fast(sid, target_id, source_mask, component_name, subsample_step=subsample_step)
                if transferred_mask is not None:
                    projected_masks.append(transferred_mask)
        
        if len(projected_masks) < len(source_ids):
            print(" Projection failed.")
            continue
            
        target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
        union_mask = np.zeros((target_h, target_w), dtype=np.uint8)
        for mask in projected_masks:
            union_mask = cv2.bitwise_or(union_mask, mask)
        
        iou_score = calculate_iou(union_mask, target_gt_mask)
        iou_scores.append(iou_score)
        print(f" IoU: {iou_score:.4f}")

        # --- THIS IS THE RESTORED VISUALIZATION LOGIC ---
        if iou_score < low_iou_threshold:
            print(f"    -> Low IoU detected! Saving failure case image...")
            target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
            
            num_sources = len(projected_masks)
            # Create a 3-row plot to show everything
            fig, axes = plt.subplots(3, num_sources, figsize=(10 * num_sources, 30))
            fig.suptitle(f"Low IoU ({iou_score:.4f}) Analysis for {component_name} on {images[target_id].name}", fontsize=30, y=0.95)
            
            # Row 1: Source Masks
            for j in range(num_sources):
                axes[0, j].imshow(source_rgbs_for_viz[j]); 
                axes[0, j].imshow(np.ma.masked_where(source_masks_for_viz[j]==0, source_masks_for_viz[j]), cmap='cool', alpha=0.6)
                axes[0, j].set_title(f"Source {chr(65+j)}"); 
                axes[0, j].axis('off')

            # Row 2: Individual Projections
            for j in range(num_sources):
                axes[1, j].imshow(target_rgb); 
                axes[1, j].imshow(np.ma.masked_where(projected_masks[j]==0, projected_masks[j]), cmap='plasma', alpha=0.6)
                axes[1, j].set_title(f"Projection from Source {chr(65+j)}"); 
                axes[1, j].axis('off')

            # Row 3: Final Union vs. Ground Truth
            axes[2, 0].imshow(target_rgb)
            overlay_res = target_rgb.copy()
            roi_res = overlay_res[union_mask > 0]
            blended_res = (roi_res*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8)
            overlay_res[union_mask > 0] = blended_res
            axes[2, 0].imshow(overlay_res)
            axes[2, 0].set_title("Final Union Result", fontsize=18); 
            axes[2, 0].axis('off')
            
            axes[2, 1].imshow(target_rgb)
            overlay_gt = target_rgb.copy()
            roi_gt = overlay_gt[target_gt_mask > 0]
            blended_gt = (roi_gt*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8)
            overlay_gt[target_gt_mask > 0] = blended_gt
            axes[2, 1].imshow(overlay_gt)
            axes[2, 1].set_title("Ground Truth", fontsize=18); 
            axes[2, 1].axis('off')

            # Hide any unused axes in the bottom row
            for j in range(2, num_sources):
                axes[2, j].axis('off')

            save_path = os.path.join(output_folder, f"failure_{images[target_id].name}_iou_{iou_score:.4f}.png")
            plt.tight_layout(rect=[0, 0, 1, 0.93])
            plt.savefig(save_path)
            plt.close(fig) # Close the figure to prevent display in the notebook

    end_time = time.time()
    total_time = end_time - start_time
    
    return iou_scores, total_time

In [ ]:
# *** CONTROL PANEL ***
component_to_analyze = "bumper_f/cover" 
low_iou_threshold = 0.65
# *********************

# --- Setup file paths ---
sanitized_name = component_to_analyze.replace("/", "_")
report_file_path = f'{sanitized_name}_analysis_report.txt'
results_file_path = f'{sanitized_name}_analysis_results.json'
subsample_steps_to_test = [7, 8, 9]

# --- Load or initialize results ---
if os.path.exists(results_file_path):
    with open(results_file_path, 'r') as f:
        all_results = json.load(f)
else:
    all_results = {}

# --- Run Analysis ---
for step in subsample_steps_to_test:
    step_str = str(step)
    if step_str not in all_results:
        scores, total_time = run_component_analysis(component_to_analyze, subsample_step=step, low_iou_threshold=low_iou_threshold)
        if scores:
            all_results[step_str] = {"iou_scores": scores, "time": total_time}
            with open(results_file_path, 'w') as f:
                json.dump(all_results, f, indent=4)
    else:
        print(f"\nAnalysis for subsample_step={step} already complete. Skipping.")

# --- Generate and Save the Final Report ---
print(f"\n\n" + "="*40)
print(f"      ANALYSIS REPORT FOR: '{component_to_analyze.upper()}'")
print("="*40)
report_lines = []
for step, data in all_results.items():
    avg_iou = np.mean(data["iou_scores"])
    report_lines.append(f"\n--- Subsample Step: {step}x{step} ---")
    report_lines.append(f"  - Total Time: {data['time']:.2f} seconds")
    report_lines.append(f"  >>> Average IoU: {avg_iou:.4f}")
final_report_string = "\n".join(report_lines)
with open(report_file_path, 'w') as f_out:
    f_out.write(final_report_string)
print(final_report_string)

# --- Generate Final Histogram for IoU ---
if all_results:
    print("\n\n" + "="*40)
    print("      IOU SCORE DISTRIBUTION")
    print("="*40)
    
    fig, ax = plt.subplots(figsize=(15, 8))
    
    data_to_plot = [all_results[step]["iou_scores"] for step in sorted(all_results.keys())]
    labels = [f'Step {step}x{step}' for step in sorted(all_results.keys())]
    
    ax.hist(data_to_plot, bins=20, label=labels, range=(0,1), density=True)
    ax.set_title(f"Distribution of IoU Scores for '{component_to_analyze}' Projection", fontsize=18)
    ax.set_xlabel('Intersection over Union (IoU)', fontsize=14)
    ax.set_ylabel('Frequency Density', fontsize=14)
    ax.legend(fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.show()

print(f"\n\nAnalysis complete. Full report saved to '{report_file_path}'.")

## Phase 12: Tuning Section (Adjust the threshold and make a better version of v3_fast)


In [ ]:
def transfer_mask_v5(
    source_id, target_id, source_component_mask, component_name,
    images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path,
    subsample_step=9,
    depth_tolerance=0.20,
    kernel_size=11
):
    """
    V5 ("Tuned & Filtered"): A tunable version that also filters out
    source masks that are too small, using a component-specific threshold.
    """
    source_h, source_w, _ = cv2.imread(image_id_to_rgb_path[source_id]).shape
    mask_area = np.sum(source_component_mask > 0)
    image_area = source_h * source_w
    
    # --- Get the component-specific threshold and apply the filter ---
    min_mask_ratio = get_mask_size_threshold(component_name)
    if (mask_area / image_area) < min_mask_ratio:
        print(f"    -> Source mask is too small ({mask_area / image_area:.2%}), below threshold of {min_mask_ratio:.2%}. Skipping this view.")
        return None

    # (The internal logic is the same as your v3_fast, but uses the new tunable parameters)
    target_rgb_for_size = cv2.imread(image_id_to_rgb_path[target_id]); target_h, target_w, _ = target_rgb_for_size.shape
    source_depth_map_raw = cv2.imread(image_id_to_depth_path[source_id], cv2.IMREAD_UNCHANGED)
    inpaint_mask = np.where(source_depth_map_raw == 0, 255, 0).astype(np.uint8)
    source_depth_map = cv2.inpaint(source_depth_map_raw, inpaint_mask, 3, cv2.INPAINT_NS)
    source_depth_map = cv2.resize(source_depth_map, (source_w, source_h), interpolation=cv2.INTER_NEAREST)
    subsampled_mask = source_component_mask[::subsample_step, ::subsample_step]
    source_pixels_y_sub, source_pixels_x_sub = np.where(subsampled_mask > 0)
    source_pixels_y = source_pixels_y_sub * subsample_step
    source_pixels_x = source_pixels_x_sub * subsample_step
    source_pixels = np.vstack((source_pixels_x, source_pixels_y)).T
    source_image_model, source_camera_model = images[source_id], cameras[images[source_id].camera_id]
    point_cloud_3d = []
    for u, v in source_pixels:
        depth, _ = get_depth_from_map(u, v, source_depth_map)
        if depth:
            p_3d = back_project_point(u, v, depth, source_camera_model, source_image_model)
            if p_3d is not None: point_cloud_3d.append(p_3d)
    if not point_cloud_3d: return None
    point_cloud_3d = np.array(point_cloud_3d)
    shape_type = classify_component_shape(component_name)
    points_to_project = None
    if shape_type == 'planar':
        points_to_project = fit_plane_ransac(point_cloud_3d)
    else:
        if len(point_cloud_3d) < 4: return None
        tri = Delaunay(point_cloud_3d)
        points_to_project = point_cloud_3d[np.unique(tri.simplices)]
    if points_to_project is None or len(points_to_project) == 0: return None
    target_depth_map_raw = cv2.imread(image_id_to_depth_path[target_id], cv2.IMREAD_UNCHANGED)
    if target_depth_map_raw is None: return None
    target_depth_map = cv2.resize(target_depth_map_raw, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    visibility_checked_mask = np.zeros((target_h, target_w), dtype=np.uint8)
    target_image_model, target_camera_model = images[target_id], cameras[images[target_id].camera_id]
    R_target, t_target = qvec2rotmat(target_image_model.qvec), target_image_model.tvec
    for p_3d in points_to_project:
        p_cam_target = R_target @ p_3d + t_target
        z_projected = p_cam_target[2]
        if z_projected > 0:
            p_2d_target = project_point(p_3d, target_camera_model, target_image_model)
            if p_2d_target:
                u_t, v_t = int(round(p_2d_target[0])), int(round(p_2d_target[1]))
                if 0 <= v_t < target_h and 0 <= u_t < target_w:
                    z_ground_truth, _ = get_depth_from_map(u_t, v_t, target_depth_map)
                    if z_ground_truth and z_projected <= (z_ground_truth + depth_tolerance):
                        visibility_checked_mask[v_t, u_t] = 255
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    closed_mask = cv2.morphologyEx(visibility_checked_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    target_vehicle_mask = cv2.imread(image_id_to_mask_path.get(target_id, ''), cv2.IMREAD_GRAYSCALE)
    if target_vehicle_mask is not None:
        if target_vehicle_mask.shape != (target_h, target_w):
            target_vehicle_mask = cv2.resize(target_vehicle_mask, (target_w, target_h), interpolation=cv2.INTER_NEAREST)
        return cv2.bitwise_and(closed_mask, target_vehicle_mask)
    else:
        return closed_mask


In [ ]:
# --- 1. Main Execution Block for Failure Case Analysis ---

failure_folder_base = "low_iou_failures"
components_to_retest = ["bonnet", "bumper_f/cover"]
subsample_steps_to_retest = [7, 8, 9]
main_output_dir = "V5_Tuning_Analysis"
os.makedirs(main_output_dir, exist_ok=True)

# --- Path for the stateful results file ---
results_file_path = os.path.join(main_output_dir, "v5_tuning_results.json")

# --- Load existing results or initialize a new dictionary ---
if os.path.exists(results_file_path):
    print(f"Loading existing results from '{results_file_path}'...")
    with open(results_file_path, 'r') as f:
        tuning_results = json.load(f)
else:
    print("No existing results file found. Starting a new analysis.")
    tuning_results = {}

# This dictionary will be used for the final report summary
tuning_report_data = {}

for step in subsample_steps_to_retest:
    step_str = str(step)
    tuning_report_data[step_str] = {"failures_before": 0, "failures_fixed": 0}
    
    for component_name in components_to_retest:
        print(f"\n{'='*25}\n--- Re-testing Failures for '{component_name}' (Step {step}) ---\n{'='*25}")
        
        sanitized_name = component_name.replace('/', '_')
        failure_folder = os.path.join(failure_folder_base, sanitized_name, f"step_{step}")
        if not os.path.isdir(failure_folder):
            print(f"No failure folder found for this configuration. Skipping.")
            continue
            
        failure_files = glob.glob(os.path.join(failure_folder, 'failure_*.png'))
        if not failure_files:
            print("No failure cases found in this folder.")
            continue

        tuning_report_data[step_str]["failures_before"] += len(failure_files)

        for file_path in failure_files:
            base_name = os.path.basename(file_path)
            
            # --- NEW: Check if this file has already been processed ---
            if base_name in tuning_results:
                print(f"\n--- Analysis for {base_name} already complete. Skipping computation. ---")
                # Use the saved scores to update the report
                iou_tuned = tuning_results[base_name]["iou_tuned"]
                if iou_tuned >= 0.65:
                    tuning_report_data[step_str]["failures_fixed"] += 1
                continue # Move to the next file

            # If not skipped, proceed with the full analysis
            target_image_name = '_'.join(base_name.split('_iou_')[0].split('_')[1:])
            target_id = next((img_id for img_id, img in images.items() if img.name == target_image_name), None)
            
            if target_id is None: continue

            print(f"\n--- Re-analyzing Target: {target_image_name} ---")

            source_ids = find_corner_views(component_name, component_to_views_map, images)
            target_gt_mask, _ = get_component_mask(target_image_name, component_name, image_name_to_anns, cat_id_to_name)
            
            projected_masks_orig, projected_masks_tuned = [], []
            source_masks_for_viz, source_rgbs_for_viz = [], []
            
            for sid in source_ids:
                source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
                if source_mask is not None:
                    source_masks_for_viz.append(source_mask)
                    source_rgbs_for_viz.append(cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB))
                    
                    # Original
                    mask_orig = transfer_mask_v3_fast(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path, subsample_step=step)
                    if mask_orig is not None: projected_masks_orig.append(mask_orig)
                    # Tuned (V5)
                    mask_tuned = transfer_mask_v5(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path, subsample_step=step)
                    if mask_tuned is not None: projected_masks_tuned.append(mask_tuned)
            
            if not projected_masks_orig or not projected_masks_tuned: continue

            target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
            union_orig = np.zeros((target_h, target_w), dtype=np.uint8); union_tuned = np.zeros((target_h, target_w), dtype=np.uint8)
            for m in projected_masks_orig: union_orig = cv2.bitwise_or(union_orig, m)
            for m in projected_masks_tuned: union_tuned = cv2.bitwise_or(union_tuned, m)
            
            iou_orig = calculate_iou(union_orig, target_gt_mask)
            iou_tuned = calculate_iou(union_tuned, target_gt_mask)

            print(f"  - Original IoU: {iou_orig:.4f}")
            print(f"  - Tuned (V5) IoU: {iou_tuned:.4f}")
            if iou_tuned > iou_orig and iou_tuned >= 0.65:
                print("  >>> Improvement Detected! This case is now considered a success. <<<")
                tuning_report_data[step_str]["failures_fixed"] += 1
            
            # --- Detailed Visualization ---
            num_sources = len(source_masks_for_viz)
            fig, axes = plt.subplots(4, num_sources, figsize=(10 * num_sources, 40))
            fig.suptitle(f"Tuning Comparison for '{component_name}' on {target_image_name}", fontsize=30, y=0.95)
            target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)

            for j in range(num_sources):
                axes[0, j].imshow(source_rgbs_for_viz[j]); axes[0, j].imshow(np.ma.masked_where(source_masks_for_viz[j]==0, source_masks_for_viz[j]), cmap='cool', alpha=0.6)
                axes[0, j].set_title(f"Source {chr(65+j)}"); axes[0, j].axis('off')
            
            for j in range(num_sources):
                axes[1, j].imshow(target_rgb); axes[1, j].imshow(np.ma.masked_where(projected_masks_orig[j]==0, projected_masks_orig[j]), cmap='Reds', alpha=0.6)
                axes[1, j].set_title(f"Untuned Projection from {chr(65+j)}"); axes[1, j].axis('off')
            
            for j in range(num_sources):
                axes[2, j].imshow(target_rgb); axes[2, j].imshow(np.ma.masked_where(projected_masks_tuned[j]==0, projected_masks_tuned[j]), cmap='Greens', alpha=0.6)
                axes[2, j].set_title(f"Tuned (V5) Projection from {chr(65+j)}"); axes[2, j].axis('off')
            
            # Hide unused axes for the top 3 rows
            for row in range(3):
                for j in range(num_sources, 4):
                    if j < axes.shape[1]: axes[row, j].axis('off')

            axes[3, 0].imshow(target_rgb); overlay_orig = target_rgb.copy(); roi_o = overlay_orig[union_orig > 0]; blended_o = (roi_o*0.5+np.array([255,0,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_orig[union_orig > 0] = blended_o; axes[3, 0].imshow(overlay_orig)
            axes[3, 0].set_title(f"Untuned Union (IoU: {iou_orig:.4f})"); axes[3, 0].axis('off')
            
            axes[3, 1].imshow(target_rgb); overlay_tuned = target_rgb.copy(); roi_t = overlay_tuned[union_tuned > 0]; blended_t = (roi_t*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_tuned[union_tuned > 0] = blended_t; axes[3, 1].imshow(overlay_tuned)
            axes[3, 1].set_title(f"Tuned (V5) Union (IoU: {iou_tuned:.4f})"); axes[3, 1].axis('off')
            
            axes[3, 2].imshow(target_rgb); overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes[3, 2].imshow(overlay_gt)
            axes[3, 2].set_title("Ground Truth"); axes[3, 2].axis('off')
            
            for j in range(3, num_sources):
                if j < axes.shape[1]: axes[3, j].axis('off')

            # --- Save the figure to the new directory structure ---
            component_output_dir = os.path.join(main_output_dir, sanitized_name)
            os.makedirs(component_output_dir, exist_ok=True)
            save_path = os.path.join(component_output_dir, f"tuned_comparison_{target_image_name}_step_{step}.png")
            
            plt.tight_layout(rect=[0, 0, 1, 0.93])
            plt.savefig(save_path)
            plt.close(fig) # Close figure to prevent display
            
            # --- Save the new result immediately ---
            tuning_results[base_name] = {"iou_orig": iou_orig, "iou_tuned": iou_tuned}
            with open(results_file_path, 'w') as f:
                json.dump(tuning_results, f, indent=4)
            print(f"    - Saved results for this target to file.")

# --- 4. Final Summary Report ---
report_lines = ["="*50, "      FINAL TUNING IMPROVEMENT REPORT", "="*50]
for step, data in tuning_report_data.items():
    total_failures = data["failures_before"]
    fixed_failures = data["failures_fixed"]
    if total_failures > 0:
        fix_rate = (fixed_failures / total_failures) * 100
        report_lines.append(f"\n--- Subsample Step: {step}x{step} ---")
        report_lines.append(f"  - Initial number of failure cases: {total_failures}")
        report_lines.append(f"  - Number of cases fixed by tuning: {fixed_failures}")
        report_lines.append(f"  >>> Fix Rate: {fix_rate:.2f}% <<<")
    else:
        print(f"\n--- Subsample Step: {step}x{step} ---")
        print("  - No failure cases were found to re-test.")

final_report_string = "\n".join(report_lines)
report_file_path = os.path.join(main_output_dir, "final_tuning_report.txt")
with open(report_file_path, 'w') as f_out:
    f_out.write(final_report_string)

print("\n\n" + final_report_string)
print(f"\n\nAll failure case re-testing is complete. Visualizations and report saved in '{main_output_dir}'.")

## Run v5 on bumper and bonnet

In [ ]:
# # --- Control Panel ---
# main_output_dir = "Final_Analysis"
# results_file_path = os.path.join(main_output_dir, "final_analysis_results.json")
# report_file_path = os.path.join(main_output_dir, "final_analysis_report.txt")

# components_to_analyze = ["bonnet", "bumper_f/cover"]
# # --------------------

# # --- Load existing results or initialize a new dictionary ---
# os.makedirs(main_output_dir, exist_ok=True)
# if os.path.exists(results_file_path):
#     print(f"Loading existing results from '{results_file_path}'...")
#     with open(results_file_path, 'r') as f:
#         all_results = json.load(f)
# else:
#     print("No existing results file found. Starting a new analysis.")
#     all_results = {}

# # --- Main Analysis Loop ---
# for component_name in components_to_analyze:
#     print(f"\n{'='*25}\n--- Preparing Analysis for '{component_name}' ---\n{'='*25}")

#     if component_name not in all_results:
#         all_results[component_name] = {}

#     source_ids = find_corner_views(component_name, component_to_views_map, images)
#     if len(source_ids) < 2:
#         print(f"Not enough diverse corner views found for '{component_name}'. Skipping.")
#         continue

#     potential_targets = [tid for tid in valid_image_ids if tid not in source_ids and get_component_mask(images[tid].name, component_name, image_name_to_anns, cat_id_to_name)[0] is not None]
    
#     valid_targets = []
#     min_mask_ratio = get_mask_size_threshold(component_name)
#     for tid in potential_targets:
#         h, w, _ = cv2.imread(image_id_to_rgb_path[tid]).shape
#         gt_mask, _ = get_component_mask(images[tid].name, component_name, image_name_to_anns, cat_id_to_name)
#         if gt_mask is not None and (np.sum(gt_mask > 0) / (h * w)) >= min_mask_ratio:
#             valid_targets.append(tid)

#     if not valid_targets:
#         print(f"No valid target images with large enough ground truth found. Skipping.")
#         continue
    
#     print(f"Found {len(valid_targets)} valid targets to test against.")
    
#     # --- Loop through all valid targets, skipping completed ones ---
#     for idx, target_id in enumerate(valid_targets):
#         target_name = images[target_id].name
        
#         if target_name in all_results[component_name]:
#             print(f"  ({idx+1}/{len(valid_targets)}) Result for Target: {target_name} already exists. Skipping.")
#             continue
        
#         print(f"  ({idx+1}/{len(valid_targets)}) Testing Target: {target_name}...", end='', flush=True)
        
#         start_time = time.time()
#         projected_masks = []
#         for sid in source_ids:
#             source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
#             if source_mask is not None:
#                 transferred_mask = transfer_mask_v5(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
#                 if transferred_mask is not None:
#                     projected_masks.append(transferred_mask)
        
#         if len(projected_masks) < len(source_ids):
#             print(" Projection failed.")
#             continue
        
#         target_h, target_w, _ = cv2.imread(image_id_to_rgb_path[target_id]).shape
#         union_mask = np.zeros((target_h, target_w), dtype=np.uint8)
#         for mask in projected_masks:
#             union_mask = cv2.bitwise_or(union_mask, mask)
        
#         target_gt_mask, _ = get_component_mask(target_name, component_name, image_name_to_anns, cat_id_to_name)
#         iou_score = calculate_iou(union_mask, target_gt_mask)
#         end_time = time.time()
        
#         all_results[component_name][target_name] = {"iou": iou_score, "time": end_time - start_time}
#         with open(results_file_path, 'w') as f:
#             json.dump(all_results, f, indent=4)
        
#         print(f" IoU: {iou_score:.4f}")

# # --- 3. Final Report Generation ---
# print("\n\n" + "="*50)
# print("      FINAL EXHAUSTIVE ANALYSIS REPORT")
# print("="*50)
# report_lines = []
# for component, data in all_results.items():
#     iou_scores = [res["iou"] for res in data.values()]
#     time_scores = [res["time"] for res in data.values()]
#     avg_iou = np.mean(iou_scores)
#     avg_time = np.mean(time_scores)
    
#     report_lines.append(f"\n--- Component: '{component}' ---")
#     report_lines.append(f"  - Combinations Tested: {len(iou_scores)}")
#     report_lines.append(f"  >>> Average IoU: {avg_iou:.4f}")
#     report_lines.append(f"  >>> Average Time per Projection: {avg_time:.2f} seconds")
# final_report_string = "\n".join(report_lines)
# with open(report_file_path, 'w') as f_out:
#     f_out.write(final_report_string)
# print(final_report_string)

# # --- 4. Visualization of 20 Random Samples ---
# print("\n\n" + "="*50)
# print("      GENERATING VISUALIZATIONS FOR RANDOM SAMPLES")
# print("="*50)

# for component_name, data in all_results.items():
#     print(f"\n--- Generating visualizations for '{component_name}' ---")
    
#     # Get a list of all successfully tested targets
#     tested_targets = list(data.keys())
    
#     # Select up to 20 random targets to visualize
#     targets_to_visualize = random.sample(tested_targets, min(20, len(tested_targets)))
    
#     # Create the output folder
#     component_output_dir = os.path.join(main_output_dir, component_name.replace('/', '_'))
#     os.makedirs(component_output_dir, exist_ok=True)
    
#     print(f"Saving {len(targets_to_visualize)} plots to '{component_output_dir}'...")
    
#     for target_name in targets_to_visualize:
#         target_id = next((img_id for img_id, img in images.items() if img.name == target_name), None)
#         if target_id is None: continue

#         # We need to re-run the projection to get the intermediate masks for plotting
#         source_ids = find_corner_views(component_name, component_to_views_map, images)
#         projected_masks, source_masks_for_viz, source_rgbs_for_viz = [], [], []
#         for sid in source_ids:
#             source_mask, _ = get_component_mask(images[sid].name, component_name, image_name_to_anns, cat_id_to_name)
#             if source_mask is not None:
#                 source_masks_for_viz.append(source_mask)
#                 source_rgbs_for_viz.append(cv2.cvtColor(cv2.imread(image_id_to_rgb_path[sid]), cv2.COLOR_BGR2RGB))
#                 transferred_mask = transfer_mask_v5(sid, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path)
#                 if transferred_mask is not None:
#                     projected_masks.append(transferred_mask)
        
#         if len(projected_masks) < len(source_ids): continue
            
#         # --- Create the plot ---
#         num_sources = len(source_masks_for_viz)
#         fig, axes = plt.subplots(3, num_sources, figsize=(10 * num_sources, 30))
#         fig.suptitle(f"4-Corner Analysis for '{component_name}' on {target_name}", fontsize=30, y=0.95)
#         target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
        
#         # Row 1: Source Masks
#         for j in range(num_sources):
#             axes[0, j].imshow(source_rgbs_for_viz[j]); axes[0, j].imshow(np.ma.masked_where(source_masks_for_viz[j]==0, source_masks_for_viz[j]), cmap='cool', alpha=0.6)
#             axes[0, j].set_title(f"Source {chr(65+j)}"); axes[0, j].axis('off')

#         # Row 2: Individual Projected Masks
#         for j in range(num_sources):
#             axes[1, j].imshow(target_rgb); axes[1, j].imshow(np.ma.masked_where(projected_masks[j]==0, projected_masks[j]), cmap='plasma', alpha=0.6)
#             axes[1, j].set_title(f"Projection from Source {chr(65+j)}"); axes[1, j].axis('off')

#         # Row 3: Final Combined Results
#         union_mask = np.zeros(projected_masks[0].shape, dtype=np.uint8)
#         for mask in projected_masks: union_mask = cv2.bitwise_or(union_mask, mask)
#         target_gt_mask, _ = get_component_mask(target_name, component_name, image_name_to_anns, cat_id_to_name)
#         iou = calculate_iou(union_mask, target_gt_mask)

#         axes[2, 0].imshow(target_rgb); overlay_u = target_rgb.copy(); roi_u = overlay_u[union_mask > 0]; blended_u = (roi_u*0.5+np.array([0,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_u[union_mask > 0] = blended_u; axes[2, 0].imshow(overlay_u)
#         axes[2, 0].set_title(f"Final Union (IoU: {iou:.4f})"); axes[2, 0].axis('off')
        
#         axes[2, 1].imshow(target_rgb); overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt*0.5+np.array([255,255,0],dtype=np.uint8)*0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes[2, 1].imshow(overlay_gt)
#         axes[2, 1].set_title("Ground Truth"); axes[2, 1].axis('off')
        
#         # Hide unused axes
#         for j in range(2, num_sources):
#             axes[2, j].axis('off')

#         save_path = os.path.join(component_output_dir, f"result_{target_name}_iou_{iou:.4f}.png")
#         plt.tight_layout(rect=[0, 0, 1, 0.93])
#         plt.savefig(save_path)
#         plt.close(fig)

# print(f"\n\nAnalysis complete. Visualizations saved in '{main_output_dir}'. Full report saved to '{report_file_path}'.")

In [ ]:
# # --- Control Panel ---
# component_to_tune = "bonnet"
# subsample_step_for_tuning = 9
# num_combinations = 10 # Test on 10 random samples
# tolerances_to_test = [0.10, 0.15, 0.20, 0.30, 0.40, 0.50, 0.6] # Test 10, 15, 20, 30, and 40cm
# # -------------------

# # --- Setup file paths ---
# report_file_path = f'tolerance_tuning_report_for_{component_to_tune}.txt'
# results_file_path = f'tolerance_tuning_results_for_{component_to_tune}.json'

# # --- Load or initialize results ---
# if os.path.exists(results_file_path):
#     with open(results_file_path, 'r') as f:
#         all_results = json.load(f)
# else:
#     all_results = {}

# # --- Find a fixed set of target images ---
# print(f"\n--- Preparing to tune '{component_to_tune}' with subsample_step={subsample_step_for_tuning} ---")
# source_id = component_area_map.get(component_to_tune, [])[0][1]
# valid_targets = [i for i in valid_image_ids if i != source_id and get_component_mask(images[i].name, component_to_tune, image_name_to_anns, cat_id_to_name)[0] is not None]

# if not valid_targets or source_id is None:
#     print("Could not find enough valid data to run the tuning experiment. Stopping.")
# else:
#     targets_to_test = random.sample(valid_targets, min(num_combinations, len(valid_targets)))
#     print(f"Selected {len(targets_to_test)} random target images for a direct comparison.")

#     # --- Run Analysis for each tolerance ---
#     for tol in tolerances_to_test:
#         tol_str = f"{tol:.4f}"
#         if tol_str not in all_results:
#             print(f"\nTesting tolerance: {tol*100:.1f}cm...")
#             scores, total_time = run_tolerance_analysis(component_to_tune, subsample_step=subsample_step_for_tuning, depth_tolerance=tol, targets_to_test=targets_to_test)
#             if scores:
#                 all_results[tol_str] = {"avg_iou": np.mean(scores), "time": total_time}
#                 with open(results_file_path, 'w') as f:
#                     json.dump(all_results, f, indent=4)
#         else:
#             print(f"\nAnalysis for depth_tolerance={tol*100:.1f}cm already complete. Skipping.")

#     # --- Generate and Save the Final Report ---
#     print("\n\n" + "="*50)
#     print(f"      FINAL DEPTH TOLERANCE REPORT FOR '{component_to_tune.upper()}'")
#     print("="*50)
#     report_lines = []
#     for tol_str, data in sorted(all_results.items(), key=lambda item: float(item[0])):
#         tolerance_cm = float(tol_str) * 100
#         report_lines.append(f"  - Tolerance: {tolerance_cm:4.1f}cm | Avg IoU: {data['avg_iou']:.4f} | Total Time: {data['time']:.2f}s")
    
#     final_report_string = "\n".join(report_lines)
#     with open(report_file_path, 'w') as f_out:
#         f_out.write(final_report_string)
#     print(final_report_string)

#     # --- Generate Final Plot ---
#     if all_results:
#         print("\n\n" + "="*50)
#         print("      IOU VS. DEPTH TOLERANCE")
#         print("="*50)
        
#         sorted_tolerances = sorted([float(t) for t in all_results.keys()])
#         iou_values = [all_results[f"{t:.4f}"]["avg_iou"] for t in sorted_tolerances]
        
#         best_iou = np.max(iou_values)
#         best_tolerance = sorted_tolerances[np.argmax(iou_values)]

#         fig, ax = plt.subplots(figsize=(15, 8))
#         ax.plot(np.array(sorted_tolerances) * 100, iou_values, marker='o', linestyle='--')
        
#         ax.axvline(best_tolerance * 100, color='r', linestyle='--', label=f'Optimal Tolerance ({best_tolerance*100:.1f}cm)')
#         ax.axhline(best_iou, color='g', linestyle='--', label=f'Best IoU ({best_iou:.4f})')
        
#         ax.set_title(f"Effect of Depth Tolerance on IoU for '{component_to_tune}'", fontsize=18)
#         ax.set_xlabel("Depth Tolerance (cm)", fontsize=14)
#         ax.set_ylabel("Average Intersection over Union (IoU)", fontsize=14)
#         ax.grid(True)
#         ax.legend(fontsize=12)
        
#         plt.show()

#     print(f"\n\nAnalysis complete. Full report saved to '{report_file_path}'.")

## Final Section: Analyse the performance of all components with 4 angles around the car

In [ ]:
# --- 1. Define the 3 Fixed Examples for Comparison ---
try:
    fixed_examples = [
        {
            "component": "bonnet",
            "source_id": component_to_views_map["bonnet"][0][1], # Best view
            "target_id": component_to_views_map["bonnet"][5][1]  # A different, good view
        },
        {
            "component": "windshield_f",
            "source_id": component_to_views_map["windshield_f"][0][1], # Best view
            "target_id": component_to_views_map["windshield_f"][3][1]   # A different, good view
        },
        {
            "component": "bumper_f/cover",
            "source_id": component_to_views_map["bumper_f/cover"][0][1], # Best view
            "target_id": component_to_views_map["bumper_f/cover"][7][1] # A different, good view
        }
    ]
except (KeyError, IndexError):
    print("Warning: Could not create fixed examples. Falling back to a single random example.")
    # Fallback to a single random example if the fixed ones aren't available
    component_name = random.choice(["bonnet", "bumper_f/cover", "windshield_f"])
    source_id = component_to_views_map.get(component_name, [])[0][1]
    target_id = random.choice([i for i in valid_image_ids if i != source_id and get_component_mask_robust(images[i].name, component_name, image_name_to_anns, cat_id_to_name, images[i].height, images[i].width)[0] is not None])
    fixed_examples = [{"component": component_name, "source_id": source_id, "target_id": target_id}]


# --- 2. Main Execution Block (Modified) ---
for i, example in enumerate(fixed_examples):
    component_name = example["component"]
    source_id = example["source_id"]
    target_id = example["target_id"]
    
    print(f"\n{'='*25}\n--- Running Comparison Example {i+1} for '{component_name}' ---\n{'='*25}")
    print(f"  - Source: {images[source_id].name}")
    print(f"  - Target: {images[target_id].name}")

    # --- Get Source and Target Masks ---
    source_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[source_id]), cv2.COLOR_BGR2RGB)
    source_h, source_w, _ = source_rgb.shape
    source_mask, _ = get_component_mask_robust(images[source_id].name, component_name, image_name_to_anns, cat_id_to_name, source_h, source_w)
    
    target_rgb = cv2.cvtColor(cv2.imread(image_id_to_rgb_path[target_id]), cv2.COLOR_BGR2RGB)
    target_h, target_w, _ = target_rgb.shape
    target_gt_mask, _ = get_component_mask_robust(images[target_id].name, component_name, image_name_to_anns, cat_id_to_name, target_h, target_w)

    if source_mask is None or target_gt_mask is None:
        print("Could not get a valid source or target mask for this example. Skipping.")
        continue

    # --- Run All Transfer Methods, Calculate IoU, and Time ---
    masks = {}
    ious = {}
    times = {}
    
    print("\n--- Running Methods ---")
    
    # V1
    start_time = time.time(); masks['v1'] = transfer_mask_v1(source_id, target_id, source_mask, images, cameras, image_id_to_rgb_path, image_id_to_depth_path); times['v1'] = time.time() - start_time
    ious['v1'] = calculate_iou(masks['v1'], target_gt_mask)
    
    # V2
    start_time = time.time(); masks['v2'] = transfer_mask_v2(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path); times['v2'] = time.time() - start_time
    ious['v2'] = calculate_iou(masks['v2'], target_gt_mask)
    
    # V3
    start_time = time.time(); masks['v3'] = transfer_mask_v3(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v3'] = time.time() - start_time
    ious['v3'] = calculate_iou(masks['v3'], target_gt_mask)
    
    # --- NEW: V3_FAST ADDED HERE ---
    start_time = time.time(); masks['v3_fast'] = transfer_mask_v3_fast(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v3_fast'] = time.time() - start_time
    ious['v3_fast'] = calculate_iou(masks['v3_fast'], target_gt_mask)
    
    # V4
    start_time = time.time(); masks['v4'] = transfer_mask_v4(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v4'] = time.time() - start_time
    ious['v4'] = calculate_iou(masks['v4'], target_gt_mask)
    
    # V5
    start_time = time.time(); masks['v5'] = transfer_mask_v5(source_id, target_id, source_mask, component_name, images, cameras, image_id_to_rgb_path, image_id_to_depth_path, image_id_to_mask_path); times['v5'] = time.time() - start_time
    ious['v5'] = calculate_iou(masks['v5'], target_gt_mask)

    # --- UPDATED LIST OF VERSIONS ---
    versions_to_run = ['v1', 'v2', 'v3', 'v3_fast', 'v4', 'v5']

    print("\n--- Quantitative Results ---")
    for version in versions_to_run:
        print(f"  - {version.upper():<8}: Time = {times[version]:.2f}s | IoU = {ious[version]:.4f}")

    # --- Separate Visualization for Each Method ---
    for version in versions_to_run:
        fig, axes = plt.subplots(1, 3, figsize=(30, 10))
        fig.suptitle(f"Method {version.upper()} Comparison for '{component_name}'", fontsize=24)

        # Panel 1: Source
        axes[0].imshow(source_rgb); axes[0].imshow(np.ma.masked_where(source_mask==0, source_mask), cmap='cool', alpha=0.6)
        axes[0].set_title("1. Source Mask", fontsize=18); axes[0].axis('off')

        # Panel 2: Transferred Result
        axes[1].imshow(target_rgb)
        if masks[version] is not None:
            overlay_res = target_rgb.copy(); roi_res = overlay_res[masks[version] > 0]; blended_res = (roi_res * 0.5 + np.array([0, 255, 0], dtype=np.uint8) * 0.5).astype(np.uint8); overlay_res[masks[version] > 0] = blended_res; axes[1].imshow(overlay_res)
        axes[1].set_title(f"2. {version.upper()} Result\n(Time: {times[version]:.2f}s | IoU: {ious[version]:.4f})", fontsize=18)
        axes[1].axis('off')

        # Panel 3: Ground Truth
        axes[2].imshow(target_rgb); overlay_gt = target_rgb.copy(); roi_gt = overlay_gt[target_gt_mask > 0]; blended_gt = (roi_gt * 0.5 + np.array([255, 255, 0], dtype=np.uint8) * 0.5).astype(np.uint8); overlay_gt[target_gt_mask > 0] = blended_gt; axes[2].imshow(overlay_gt)
        axes[2].set_title("3. Ground Truth Mask", fontsize=18); axes[2].axis('off')

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

print("\n\nAll comparison examples are complete.")